In [2515]:
import requests
import re
import sys
import time
import pandas as pd
from langchain_chroma import Chroma
from langchain_core.example_selectors import MaxMarginalRelevanceExampleSelector
from langchain_ollama import OllamaEmbeddings
from ollama import Client
import json
import subprocess
import time
import ipaddress
from datetime import datetime
import shlex
import math
from typing import Optional, Dict, Any
import argparse, secrets, hashlib

In [2516]:
RYU_CONTROLLER_IP = "127.0.0.1"
RYU_CONTROLLER_PORT = 8080
url = f"http://{RYU_CONTROLLER_IP}:{RYU_CONTROLLER_PORT}/stats/flowentry/delete"
# ONOS Controller Details
Ryu_BASE_URL = f"http://{RYU_CONTROLLER_IP}:{RYU_CONTROLLER_PORT}/stats/flowentry/"
# Define sudo password
sudo_password = "test@irciss008" #your localhost password

In [2517]:
ip_to_host = {
    "10.0.1.1": "h1",
    "10.0.1.2": "h2",
    "10.0.1.3": "h3",
    "10.0.1.4": "h4"
            }

host_to_ip = {
    "h1": "10.0.1.1",
    "h2": "10.0.1.2",
    "h3": "10.0.1.3",
    "h4": "10.0.1.4",
}

# --- Diamond topology wiring helpers (drop-in) ---

SW_OF = {
    "1": "of:0000000000000001",  # s1
    "2": "of:0000000000000002",  # s2
    "3": "of:0000000000000003",  # s3
    "4": "of:0000000000000004",  # s4
}

HOSTS = {
    "h1": "10.0.1.1",
    "h2": "10.0.1.2",
    "h3": "10.0.1.3",
    "h4": "10.0.1.4",
}

# Host attachment (edge switch, access port) from your Mininet build
HOST_ATTACH = {
    HOSTS["h1"]: (SW_OF["1"], 3),  # h1 -> s1:3
    HOSTS["h2"]: (SW_OF["1"], 4),  # h2 -> s1:4
    HOSTS["h3"]: (SW_OF["4"], 3),  # h3 -> s4:3
    HOSTS["h4"]: (SW_OF["4"], 4),  # h4 -> s4:4
}

In [2518]:
switch_id_for_llm_assurance = None
llm_caller_flag = 0
gl_cookie = 0

In [2519]:
def delete_flow(base, flow):
    """
    Deletes a specific flow rule from a switch using OFPFC_DELETE_STRICT.
    'flow' must be a valid Ryu flow rule dictionary.
    """
    # Use OFPFC_DELETE_STRICT to delete the exact flow
    # We must include match, priority, and cookie (if it exists)
    payload = {
        "dpid": flow.get("dpid"),
        "cmd": "OFPFC_DELETE_STRICT",
        "table_id": flow.get("table_id", 0),
        "priority": flow.get("priority", 0),
        "cookie": flow.get("cookie", 0),
        "cookie_mask": flow.get("cookie_mask", 0) if flow.get("cookie") else 0,
        "match": flow.get("match", {})
    }
    
    # If a cookie was specified, use a full mask for an exact match
    if payload["cookie"] != 0:
        payload["cookie_mask"] = 0xffffffffffffffff

    r = requests.post(f"{base}/stats/flowentry/delete", json=payload, timeout=5)
    if r.status_code != 200:
        print(f"Warning: Delete failed [{r.status_code}]: {r.text}")
        return False
    return True

def ryu_get(base, path):
    r = requests.get(f"{base}{path}", timeout=5)
    r.raise_for_status()
    return r.json()

def wait_for_dpids(base, expect, timeout=30):
    want = set(expect); start = time.time()
    seen = set()
    while time.time() - start < timeout:
        try:
            seen = set(ryu_get(base, "/stats/switches"))
            if want.issubset(seen):
                return sorted(seen)
        except requests.RequestException:
            pass
        time.sleep(1)
    return sorted(seen)

def add_flow(base, flow):
    r = requests.post(f"{base}/stats/flowentry/add", json=flow, timeout=5)
    if r.status_code != 200:
        raise SystemExit(f"Add failed [{r.status_code}]: {r.text}")
    return True

def get_flows(base, dpid):
    return ryu_get(base, f"/stats/flow/{dpid}")


def find_by_cookie(flow_dump, cookie):
    for f in flow_dump:
        if int(f.get("cookie", -1)) == int(cookie):
            return f
    return None

def roughly_matches(entry, want_match, want_prio):
    def norm(m):
        m = dict(m)
        if "dl_type" in m and "eth_type" not in m: m["eth_type"] = m.pop("dl_type")
        if "nw_proto" in m and "ip_proto" not in m: m["ip_proto"] = m.pop("nw_proto")
        return m
    return entry.get("priority") == want_prio and norm(entry.get("match", {})) == norm(want_match)

def exists(base, dpid, match, priority, cookie=None, retries=3, delay=0.5):
    for _ in range(retries):
        dump = get_flows(base, dpid).get(str(dpid), [])
        if cookie is not None:
            # Cookie-only check: if not found, do NOT fall back to match/priority
            found = find_by_cookie(dump, cookie)
            if found:
                return True, found
        else:
            # Only use match+priority when no cookie is specified
            for f in dump:
                if roughly_matches(f, match, priority):
                    return True, f
        time.sleep(delay)
    return False, None

def fingerprint(flow):
    h = hashlib.sha1(json.dumps({
        "table_id": flow.get("table_id", 0),
        "priority": flow.get("priority"),
        "match": flow.get("match"),
        "actions": flow.get("actions", [])
    }, sort_keys=True).encode()).hexdigest()
    return h[:12]

In [2520]:
TRANSLATION_PROMPT_RYU = """
You are a meticulous network engineer. Convert the user's **intent** into a single Ryu (OpenFlow) flow rule
expressed as a **JSON object only** (no comments, no code fences, no extra text).

Return exactly one JSON object with this schema (keys in lower-case snake_case):

{
  "dpid": <int>,                  // datapath ID (switch ID). Example: 1
  "table_id": <int>,              // default 0 unless the intent clearly states otherwise
  "priority": <int>,              // guidance below
  "match": {                      // match conditions
      // Common keys (use only those needed by the intent):
      "in_port": <int>,
      "eth_type": <int>,          // IPv4=2048 (0x0800), IPv6=34525 (0x86DD)
      "ip_proto": <int>,          // TCP=6, UDP=17, ICMPv4=1, ICMPv6=58
      "ipv4_src": "<ip[/mask]>",
      "ipv4_dst": "<ip[/mask]>",
      "ipv6_dst": "<ip6[/mask]>",
      "tcp_src": <int>,
      "tcp_dst": <int>,
      "udp_src": <int>,
      "udp_dst": <int>,
      "icmpv4_type": <int>        // e.g., 8 for echo request (ping)
  },
  "actions": [                    // ordered list of actions
      // For forwarding use: {"type":"OUTPUT","port":<int|IN_PORT|FLOOD|CONTROLLER|LOCAL>}
      // For queue/QoS:      {"type":"SET_QUEUE","queue_id":<int>}
      // For VLAN ops:       {"type":"PUSH_VLAN","ethertype":33024} (0x8100), 
      //                     {"type":"SET_FIELD","field":"vlan_vid","value":<int>}
      // To drop:            []   (empty list)
  ]
}

Rules of interpretation:
- "in switch N" or "on switch N" → dpid=N. If not specified, infer the most reasonable dpid from context; otherwise omit only if impossible.
- IPv4 traffic → include eth_type=2048. IPv6 traffic → include eth_type=34525.
- ICMP/ICMPv6 ping (echo request) → ip_proto=1 (or 58 for v6) AND icmpv4_type=8 when clearly IPv4.
- "HTTP" → TCP port 80; "HTTPS" → TCP port 443; "DNS" → UDP/TCP port 53 (use UDP unless otherwise stated).
- "send out/through port X", "via interface X" → OUTPUT to port X.
- "block", "drop", "deny" → actions must be an empty list [].
- QoS/priority queues → include SET_QUEUE with the specific queue_id when requested.
- table_id=0 unless the intent explicitly mentions another table.
- Priority guideline (use the highest that fits the specificity):
    300 → explicit block/deny rules
    200 → highly specific forwarding with QoS/VLAN modifications
    100 → specific L3/L4 matches (e.g., dst IP and TCP/UDP port)
     50 → catch-all/less specific fall-through (e.g., only ip_proto or in_port)

Constraints:
- Output must be a single JSON object. No surrounding text, no markdown, no explanations.
- Use **only** fields necessary to satisfy the intent; do not invent values.
- Numeric fields may be written as integers (not strings). Use lowercase key names as shown.
- If something is ambiguous, choose the most conservative, commonly-used OpenFlow interpretation for Ryu.
"""

In [2521]:
CONFLICT_PROMPT_RYU = """
You are an expert network engineer analyzing two Ryu (OpenFlow) flow rules to decide whether they **conflict**.

### **Primary Goal**
A conflict exists if the two rules apply to the **same switch (dpid)** and **same table_id**, their **matches overlap** (can match the same packet), AND at least one of the following is true:
1.  Their **actions are different** (e.g., `OUTPUT:1` vs. `OUTPUT:2`, or `[]` vs. `OUTPUT:1`).
2.  They are **redundant** (they overlap and have the **same action**).

### **Guiding Principles**
- If the `dpid` or `table_id` are different, they do **not** conflict.
- If the matches are **mutually exclusive** (e.g., they match on different L2 protocols like `LLDP` vs. `IPv4`, or different ports like `in_port: 1` vs `in_port: 2`), they do **not** conflict.

---
### **Examples**

**Example 1: Conflict (Different Action)**
Flow 1: {"dpid": 1, "priority": 100, "match": {"eth_type": 2048}, "actions": ["OUTPUT:1"]}
Flow 2: {"dpid": 1, "priority": 200, "match": {"eth_type": 2048, "ip_proto": 6}, "actions": ["OUTPUT:2"]}
Output: {
    "conflict_status": 1,
    "conflict_explanation": "Rules overlap (Flow 2 is a subset of Flow 1) but have different OUTPUT actions."
}

**Example 2: No Conflict (Mutual Exclusion)**
Flow 1: {"dpid": 4, "priority": 100, "match": {"eth_type": 2048, "ipv4_dst": "10.0.1.4"}, "actions": ["OUTPUT:4"]}
Flow 2: {"dpid": 4, "priority": 65535, "match": {"dl_dst": "01:80:c2:00:00:00"}, "actions": ["OUTPUT:CONTROLLER"]}
Output: {
    "conflict_status": 0,
    "conflict_explanation": "Matches are mutually exclusive. Flow 1 matches IPv4 packets (eth_type 2048), while Flow 2 matches a non-IPv4 L2 protocol (LLDP)."
}

**Example 3: Conflict (Redundancy / Same Action)**
Flow 1: {"dpid": 1, "priority": 100, "match": {"eth_type": 2048, "ip_proto": 6}, "actions": ["OUTPUT:1"]}
Flow 2: {"dpid": 1, "priority": 90, "match": {"eth_type": 2048}, "actions": ["OUTPUT:1"]}
Output: {
    "conflict_status": 1,
    "conflict_explanation": "Rules are redundant. The matches overlap (Flow 1 is a subset of Flow 2) and they have the same action."
}
---

### **Input Format**
You will be provided with **two JSON flow rules** in the following format:

**Flow 1:**
<JSON for Flow 1>

**Flow 2:**
<JSON for Flow 2>

### **Expected Output Format**
Respond strictly in valid JSON format, using the schema below:

{
    "conflict_status": <integer>,
    "conflict_explanation": "<conflict explanation, if any>"
}

Field Descriptions:
   - 'conflict_status' should be 1 if a conflict exists, 0 otherwise.
   - 'conflict_explanation' is a brief explanation if a conflict exists, otherwise an empty string "".

NO EXTRA TEXT, COMMENTS, OR EXPLANATIONS OUTSIDE JSON.
"""

In [2522]:
SLICING_PROMPT = """You are tasked with analyzing a natural language intent to determine if it contains a command to create or use a queue/slice in an OpenFlow switch. You should respond in JSON format.

### Rules for Interpretation:
1. **Queue/Slice Detection:**  
   - The intent is considered related to queue/slice if it contains commands such as:
     - "create queue", "create slices", "slice the network", "implement slicing", "slice the flow", "make flowspace slicing", "do slicing", "slice", "implement queue", "do queuing", "assign queue", "assign slice", or any similar phrasing.
   - If the intent does not mention creating or using a queue/slice, set the field `"use_queue"` to `0`.

2. **Switch, Queue, and Port Identification (Data-plane port/interface):**  
   - If the intent specifies a **switch ID** (e.g., "switch 4" or "openflow:4" or "node 4" or "openflow 4"), populate the `"switch_id"` field with its value.  
   - If the intent specifies a **Queue ID or slice ID** (e.g., "queue 4" or "4th queue" or "fourth queue" or "slice 1" or "first slice"), populate the `"queue_id"` field with its value.  
   - If the intent specifies a **port ID** referring to the device interface/output (e.g., "port 2" or "interface 2" or "ethernet 2" or "output node connector 2" or "second port" or "second interface"), populate the `"port_id"` field with its value. If there are multiple instances of "port_id" present, take the one which indicates the **output port or outgoing interface**.
   - If the intent does not specify a switch ID or queue ID or port ID, set the respective field to an empty string (`""`).

3. **Traffic Type and L4 Destination Port Extraction (Protocol/Service):**
   - Detect the **Layer-4 protocol** if mentioned: `"tcp"` or `"udp"`. Populate this in `"traffic_type"`. If not specified, set `"traffic_type"` to `""`.
   - Detect the **destination application port number** (Layer-4 port), if specified as a number (e.g., "port 80", "UDP port 53") or implied via a service reference such as "HTTP (TCP port 80)". Populate this number (as a string) in `"l4_port"`.  
   - Do **not** confuse the L4 port (e.g., 80/53) with the device/interface port (e.g., switch port 2). The former goes to `"l4_port"`, the latter to `"port_id"`.
   - If multiple L4 ports are mentioned, prefer the **destination/service port** used by the traffic selector (e.g., "traffic destined for port 80"). If still ambiguous, choose the first explicit destination/service port mentioned.
   - If the L4 destination port is not specified, set `"l4_port"` to `""`.

4. **Negative Constraint**: Intents that only contain commands like "block", "drop", "deny", or "forward" without any explicit mention of "queue" or "slice" are not queue-related, and use_queue must be 0.

5. **Output Format:**  
   - Respond strictly in valid JSON format adhering to the following schema:

```json
{
  "use_queue": <integer>,
  "switch_id": "<string>",
  "queue_id": "<string>",
  "port_id": "<string>",
  "traffic_type": "<string>",
  "l4_port": "<string>"
}

Field Description:
use_queue: 1 if the intent commands to create or use a queue/slice, 0 otherwise.
switch_id: Switch ID if specified in the intent, otherwise "".
queue_id: Queue/Slice ID if specified in the intent, otherwise "".
port_id: Device/interface port if specified (output/outgoing preferred), otherwise "".
traffic_type: "tcp" or "udp" if specified, otherwise "".
l4_port: Destination application port number (e.g., "80", "53") if specified, otherwise "".

No Additional Text:
Do not include any comments, explanations, or outputs outside the JSON format.

Example Inputs and Outputs:

Input Intent:
"Create a queue in switch 4 on port 3 for slicing the flow."
Output:
{
"use_queue": 1,
"switch_id": "switch 4",
"queue_id": "",
"port_id": "port 3",
"traffic_type": "",
"l4_port": ""
}

Input Intent:
"Send all video traffic through queue 0 of openflow:2."
Output:
{
"use_queue": 1,
"switch_id": "openflow 2",
"queue_id": "0",
"port_id": "",
"traffic_type": "",
"l4_port": ""
}

Input Intent:
"Configure switch 5 for traffic management."
Output:
{
"use_queue": 0,
"switch_id": "switch 5",
"queue_id": "",
"port_id": "",
"traffic_type": "",
"l4_port": ""
}

Input Intent:
"Monitor traffic flow on port 1."
Output:
{
"use_queue": 0,
"switch_id": "",
"queue_id": "",
"port_id": "port 1",
"traffic_type": "",
"l4_port": ""
}

Input Intent:
"In switch 3, if the incoming traffic in port 1 is TCP traffic destined for port 80, then pass it via interface 2, assigning it to queue 0 for prioritized handling."
Output:
{
"use_queue": 1,
"switch_id": "switch 3",
"queue_id": "0",
"port_id": "interface 2",
"traffic_type": "tcp",
"l4_port": "80"
}

Input Intent:
"For node 1, route HTTP (TCP port 80) traffic targeting 10.0.0.3/32 through port 2 with traffic assigned to queue 0."
Output:
{
"use_queue": 1,
"switch_id": "node 1",
"queue_id": "0",
"port_id": "port 2",
"traffic_type": "tcp",
"l4_port": "80"
}

Input Intent:
"Forward UDP traffic on port 53 destined for 10.0.0.9 via interface 5 of switch 2, assigning it to queue 3."
Output:
{
"use_queue": 1,
"switch_id": "switch 2",
"queue_id": "3",
"port_id": "interface 5",
"traffic_type": "udp",
"l4_port": "53"
}
"""

In [2523]:
my_models_translate_real = [
"codestral:22b",
"command-r:35b",
"huihui_ai/qwq-abliterated:latest",
]

my_models_conflict_real = [  
"huihui_ai/qwq-fusion:latest",
"qwq:latest"
]

context_examples = [3, 6]

default_model = "llama2:7b"

ollama_embedding_url = "http://localhost:11434"
ollama_server_url = "http://localhost:11435"  

ollama_emb = OllamaEmbeddings(
    model=default_model,
    base_url=ollama_embedding_url,
)

client = Client(host=ollama_server_url , timeout=120)

In [2524]:
# Load custom dataset from CSV
custom_dataset = pd.read_csv('Intent2Flow-Ryu.csv')

# Ensure proper column names and format
if not {'instruction', 'output'}.issubset(custom_dataset.columns):
    raise ValueError("The dataset must have 'instruction' and 'output' columns.")

# Split into train and test (50/50 split for example)
#trainset, testset = train_test_split(custom_dataset, test_size=0.5, random_state=42, shuffle=True)
trainset = custom_dataset

In [2525]:
def append_intent_to_store(
    file_path,
    nl_intent,
    json_flow_rule,
    device_id,
    flow_id,
    intent_type,
    intent_specificity
):
    """
    Appends an intent record to the IntentStore file in JSONL format.
    """
    record = {
        "nl_intent": nl_intent,
        "json_flow_rule": json_flow_rule,
        "device_id": device_id,
        "flow_id": flow_id,
        "intent_type": intent_type,
        "intent_specificity": intent_specificity
    }
    with open(file_path, "a", encoding="utf-8") as f:
        f.write(json.dumps(record) + "\n")

def read_intents_from_store(file_path):
    """
    Yields each intent record from the IntentStore file.
    """
    with open(file_path, "r", encoding="utf-8") as f:
        for line in f:
            intent = json.loads(line)
            yield intent

In [2526]:
def run_LLM_conflict(existing_intent_flow_json, new_intent_flow_json):
    system_prompt = CONFLICT_PROMPT_RYU

    for model in my_models_conflict_real:
        count = 0
        while True:
            count+=1
            try:
                time.sleep(0.1)

                response = client.generate(model=model,
                    options={'temperature': 0.3, 'num_ctx': 8192, 'top_p': 0.5, 'num_predict': 1024, 'num_gpu': 99},
                    stream=False,
                    system=system_prompt,
                    prompt=f"Flow 1:\n{json.dumps(existing_intent_flow_json, indent=2)}\n\nFlow 2:\n{json.dumps(new_intent_flow_json, indent=2)}",
                    format='json'
                )

                output = response['response'].strip()

                response_json = json.loads(output)

                if 'conflict_status' not in response_json:
                    #print("\nWarning: 'conflict_status' key is missing in the response.\n")
                    break
                else:
                    valid_conflict_response = True
                    conflict_status = response_json.get('conflict_status', 0)
                    # Ensure conflict_status is an integer
                    if isinstance(conflict_status, str):
                        conflict_status = int(conflict_status)

                    return valid_conflict_response, conflict_status, response_json['conflict_explanation']             

            except Exception as e:
                print("Exception found: ", e)
                sys.stdout.flush()
                if(count<15):
                    continue
                else:
                    print("\n",model, " failed to produce valid JSON for conflict info after 15 tries. Going to next model\n")
                    break               
    
    return False, None, None

In [2527]:
def conflict(device_id, new_intent_flow_json):

    # existing_flows = get_flows_for_device_ONOS(device_id) # <--- OLD

    # Use your new Ryu helper function from [cell 5]
    # Note: get_flows returns a dict like {"1": [...flows...], "2": [...]}.
    base_url = f"http://{RYU_CONTROLLER_IP}:{RYU_CONTROLLER_PORT}"
    all_flows_dict = get_flows(base_url, device_id)
    existing_flows = all_flows_dict.get(str(device_id), []) # <--- NEW

    # The rest of this function should now work, as run_LLM_conflict
    # will receive two Ryu-formatted JSON objects.
    for existing_flow in existing_flows:

        # --- START: ROBUST HARD-CODED EXCLUSION ---
        try:
            match1 = new_intent_flow_json.get("match", {})
            match2 = existing_flow.get("match", {})

            # 1. Check if a rule matches the reserved L2 control range (01:80:c2:00:00:0x)
            dl_dst_1 = str(match1.get('dl_dst', ''))
            dl_dst_2 = str(match2.get('dl_dst', ''))
            
            # This prefix precisely covers 00-0F
            is_m1_l2_special = dl_dst_1.startswith('01:80:c2:00:00:0') 
            is_m2_l2_special = dl_dst_2.startswith('01:80:c2:00:00:0')

            # 2. Check if a rule matches IPv4 or ARP
            # (eth_type can be int 2048 or hex str 0x0800)
            eth_type_1 = str(match1.get('eth_type', '')).lower()
            eth_type_2 = str(match2.get('eth_type', '')).lower()
            
            is_m1_ip_arp = eth_type_1 in ['2048', '0x0800', '2054', '0x0806'] or 'ipv4_dst' in match1 or 'ipv4_src' in match1
            is_m2_ip_arp = eth_type_2 in ['2048', '0x0800', '2054', '0x0806'] or 'ipv4_dst' in match2 or 'ipv4_src' in match2

            # 3. If one matches L2 control and the other matches IP/ARP, they are exclusive
            if (is_m1_l2_special and is_m2_ip_arp) or (is_m1_ip_arp and is_m2_l2_special):
                continue # Skip this pair, they are mutually exclusive
        
        except Exception as e:
            print(f"Warning: Pre-filter check failed: {e}")
            pass # Ignore errors and let the LLM handle it
        # --- END: ROBUST HARD-CODED EXCLUSION ---

        valid_conflict_response, conflict_status, conflict_details = run_LLM_conflict(existing_flow, new_intent_flow_json)

        if (valid_conflict_response == False):
            return 2, None, None
        elif (conflict_status == 1):
            return conflict_status, conflict_details, existing_flow

    return 0, None, None

In [2528]:
def run_LLM_IBN(intent, device_id):

    for num_examples in context_examples:
        for model in my_models_translate_real:
            
            example_selector = MaxMarginalRelevanceExampleSelector.from_examples(
                [{"instruction": trainset.iloc[0]["instruction"], "output": trainset.iloc[0]["output"]}],
                ollama_emb,
                Chroma,
                input_keys=["instruction"],
                k=num_examples,
                vectorstore_kwargs={"fetch_k": min(num_examples, len(trainset))}
                )
            # Clear and add all remaining examples from the trainset
            example_selector.vectorstore.reset_collection()
            
            for _, row in trainset.iterrows():
                example_selector.add_example({
                    "instruction": row["instruction"],
                    "output": row["output"]
                })
            
            system_prompt = TRANSLATION_PROMPT_RYU
            count = 0

            while True:
                count+=1
                try:
                    time.sleep(0.1)
                    if num_examples > 0:
                        examples = example_selector.select_examples({"instruction": intent})
                        example_str = "\n\n\n".join(map(lambda x: "Input: " + x["instruction"] + "\n\nOutput: " + x["output"], examples))
                        system_prompt += example_str + "\n\n\n"  
                    
                    response = client.generate(model=model,
                        options={'temperature': 0.6, 'num_ctx': 8192, 'top_p': 0.3, 'num_predict': 1024, 'num_gpu': 99},
                        #options={'device': 'cpu'},
                        stream=False,
                        system=system_prompt,
                        prompt=intent,
                        format='json'
                    )
                    actual_output = response['response']
                    #print("\nTranslated by: ", model)
                    break
                
                except Exception as e:
                    print("Exception on Input: ", e)
                    print("\nCheck example_str same or not: \n",example_str)
                    sys.stdout.flush()
                    if(count<15):
                        continue
                    else:
                        print("\n",model, " failed to produce valid JSON for translation info after 15 tries. Going to next model\n")
                        break 
            try:
                
                flow_json = json.loads(actual_output)
                
                #for flow in flow_json.get("flows", []):  # Iterate over all flows
                    #flow["deviceId"] = device_id  # Replace the device ID

                #print(json.dumps(flow_json))

                return flow_json
            
            except Exception as e:
                print("Exception found: ", e)

In [2529]:
def run_LLM_Slice(intent):

    system_prompt = SLICING_PROMPT
    
    for model in my_models_translate_real:     
        try:
            time.sleep(0.1)             
            response = client.generate(model=model,
                options={'temperature': 0.3, 'num_ctx': 8192, 'top_p': 0.5, 'num_predict': 1024, 'num_gpu': 99},
                #options={'device': 'cpu'},
                stream=False,
                system=system_prompt,
                prompt=intent,
                format='json'
            )
            
            output = response['response'].strip()
            response_json = json.loads(output)
            
            #print("\nCheckpoint*******Exiting LLM Slicing detection\n\n******")
            return response_json            
        
        except Exception as e:
            print("Exception found: ", e)
            sys.stdout.flush()
            continue

In [2530]:
# Add this function to [cell 5] or [cell 6]
def extract_dpid_for_ryu(intent: str) -> Optional[int]:
    """
    Extracts the integer datapath ID (dpid) for Ryu.
    """
    match = re.search(r'\b(?:switch|dpid|node|device)(?:_id)?\s*(\d+)', intent, re.IGNORECASE)
    if match:
        return int(match.group(1))
    return None # Or a default, like 1

def get_iface_for_port(device_id: int, port_no: int | str) -> str:
    """
    Builds the Mininet/OVS interface name (e.g., s4-eth1) from a
    Ryu-style integer DPID (device_id) and port number.
    
    This is a Ryu-compatible replacement and does not query any controller.
    """
    # This logic assumes your switch names are s1, s2, s3, etc.
    # which is consistent with your get_switch_name_from_device_id function
    switch_name = f"s{device_id}"
    
    return f"{switch_name}-eth{port_no}"

def classify_ryu_flow_rule(flow_rule: dict):
    """
    Classify a Ryu flow rule into a type and compute its specificity.
    
    This version correctly handles both:
    - LLM-generated actions (list of dicts): [{"type": "OUTPUT", ...}]
    - Ryu API actions (list of strings): ["OUTPUT:1"]
    
    Returns: (rule_type: str, specificity: float)
    """
    
    # --- 1. Rule Type Detection ---
    actions = flow_rule.get("actions", [])
    rule_type = "unknown" # Default

    if not actions:
        # An empty action list [] means "drop"
        rule_type = "security" 
    else:
        # --- START OF FIX ---
        # Get all unique action types, handling both str and dict
        action_types = set()
        for action in actions:
            if isinstance(action, dict):
                # Handles LLM-generated format: {"type": "OUTPUT", "port": 1}
                action_types.add(action.get("type", ""))
            elif isinstance(action, str):
                # Handles Ryu API format: "OUTPUT:1" or "OUTPUT:CONTROLLER"
                action_types.add(action.split(':', 1)[0])
        # --- END OF FIX ---
        
        if "SET_QUEUE" in action_types:
            rule_type = "qos"
        elif "OUTPUT" in action_types or "CONTROLLER" in action_types:
            # Any rule that forwards or sends to controller is "forwarding"
            rule_type = "forwarding"
        # Handle other action types if needed
        elif "GOTO_TABLE" in action_types: 
             rule_type = "forwarding"
        else:
            # Other rules (e.g., SET_FIELD, PUSH_VLAN) also fall under forwarding/modification
            rule_type = "forwarding"

    # --- 2. Specificity Computation ---
    specificity = 0.0
    match = flow_rule.get("match", {})
    
    if not match:
        return rule_type, 0.0 # Empty match has zero specificity

    for key, value in match.items():
        specificity += 1.0 # Base point for the key existing
        
        if key in ("ipv4_src", "ipv4_dst", "ipv6_src", "ipv6_dst"):
            try:
                ip_net = ipaddress.ip_network(value, strict=False) 
                if "v6" in key:
                    specificity += (ip_net.prefixlen / 128.0)
                else:
                    specificity += (ip_net.prefixlen / 32.0)
            except Exception:
                pass 
                
    return rule_type, specificity

def classify_ryu_flow_rule_old(flow_rule: dict):
    """
    Classify a Ryu flow rule into a type and compute its specificity.
    
    Returns: (rule_type: str, specificity: float)
    """
    
    # --- 1. Rule Type Detection ---
    actions = flow_rule.get("actions", [])
    rule_type = "unknown" # Default

    if not actions:
        # An empty action list [] means "drop"
        rule_type = "security" 
    else:
        # Get all unique action types, e.g., {"OUTPUT", "SET_QUEUE"}
        action_types = {action.get("type", "") for action in actions}
        
        if "SET_QUEUE" in action_types:
            rule_type = "qos"
        elif "OUTPUT" in action_types or "CONTROLLER" in action_types:
            # Any rule that forwards or sends to controller is "forwarding"
            rule_type = "forwarding"
        else:
            # Other rules (e.g., SET_FIELD, PUSH_VLAN) also fall under forwarding/modification
            rule_type = "forwarding"

    # --- 2. Specificity Computation ---
    specificity = 0.0
    match = flow_rule.get("match", {})
    
    if not match:
        return rule_type, 0.0 # Empty match has zero specificity

    for key, value in match.items():
        specificity += 1.0 # Base point for the key existing
        
        # Add extra specificity for IP masks (just like your ONOS logic)
        if key in ("ipv4_src", "ipv4_dst", "ipv6_src", "ipv6_dst"):
            try:
                # Value is like "10.0.1.1/32" or just "10.0.1.1"
                # strict=False allows single IPs
                ip_net = ipaddress.ip_network(value, strict=False) 
                
                # Add fractional specificity based on mask length
                if "v6" in key:
                    specificity += (ip_net.prefixlen / 128.0)
                else:
                    specificity += (ip_net.prefixlen / 32.0)
            except Exception:
                # If parsing fails, just count the key (already done)
                pass 
                
    return rule_type, specificity

def resolve_ryu_conflict(rule1, rule2):
    """
    Resolve conflict between two Ryu rules using Type > Specificity > Priority.
    Returns: winner_rule, loser_rule
    """
    # This priority map is the core of your policy
    type_priority = {"security": 3, "qos": 2, "forwarding": 1, "unknown": 0}

    # Get type and specificity for both rules using our new classifier
    type1, spec1 = classify_ryu_flow_rule(rule1)
    type2, spec2 = classify_ryu_flow_rule(rule2)

    # Get OpenFlow priorities (default to 0 if not present)
    p1 = rule1.get("priority", 0)
    p2 = rule2.get("priority", 0)

    # --- Resolution Hierarchy ---
    
    # 1. Type-based resolution
    type1_score = type_priority.get(type1, 0)
    type2_score = type_priority.get(type2, 0)
    
    if type1_score > type2_score:
        return rule1, rule2
    elif type2_score > type1_score:
        return rule2, rule1

    # 2. Specificity-based resolution (if types are equal)
    if spec1 > spec2:
        return rule1, rule2
    elif spec2 > spec1:
        return rule2, rule1

    # 3. Priority-based resolution (if type and specificity are equal)
    if p1 > p2:
        return rule1, rule2
    elif p2 > p1:
        return rule2, rule1

    # All are equal - this is a true tie
    return None, None

def adjust_priority_ryu(winner_rule: dict, loser_rule: dict, step: int = 10) -> dict:
    """
    Adjusts the priority of the winning Ryu flow rule so it overrides the losing one.
    This version caps the priority at 65535 to prevent overflow errors.
    """
    loser_priority = loser_rule.get("priority", 0)
    winner_priority = winner_rule.get("priority", 0)

    # Set new priority to be at least 'step' higher than the loser
    new_priority = max(winner_priority, loser_priority + step)
    
    # --- START OF FIX ---
    # Cap the new priority at the maximum allowed OpenFlow value
    if new_priority > 65535:
        print(f"Warning: Calculated priority ({new_priority}) exceeds max (65535). Capping at 65535.")
        new_priority = 65535
    # --- END OF FIX ---

    winner_rule["priority"] = new_priority
    return winner_rule


def adjust_priority_ryu_old(winner_rule: dict, loser_rule: dict, step: int = 10) -> dict:
    """
    Adjusts the priority of the winning Ryu flow rule so it overrides the losing one.
    This modifies winner_rule in-place and returns it.
    """
    loser_priority = loser_rule.get("priority", 0)
    winner_priority = winner_rule.get("priority", 0)

    # Set new priority to be at least 'step' higher than the loser
    new_priority = max(winner_priority, loser_priority + step)
    winner_rule["priority"] = new_priority

    return winner_rule

def extract_inner_flow(rule):
    return rule["flows"][0] if "flows" in rule else rule

In [2531]:
def end_to_end_IBN(intent):

    #current_time = time.time()
    device_id = extract_dpid_for_ryu(intent)

    if device_id is None:
        print("Error: Could not determine switch DPID from intent.")
        return False, None, None, None, None

    global switch_id_for_llm_assurance 
    switch_id_for_llm_assurance = device_id

    intent_JSON = run_LLM_IBN(intent, device_id)

    intent_JSON["dpid"] = device_id

    conflict_status, conflict_details, which_flow_conflict = conflict(device_id, intent_JSON)

    if((conflict_status != 1) and (conflict_status != 0)):
        print("\nCheck Conflict Detection Module, LLM did not produce a valid JSON for conflict detection.\n")
        return False, None, None, None, None

    elif (conflict_status == 1):

        winner, non_winner = resolve_ryu_conflict(intent_JSON, which_flow_conflict)

        winner_rule_inner = extract_inner_flow(winner)
        existing_rule_inner = extract_inner_flow(which_flow_conflict)

        if winner is None:
            print("\nConflict resolution resulted in a tie. The new rule and an existing rule has same type, specificity and priority\n")
            print("\nThe New flow rule:\n", intent_JSON, "\nThe existing flow rule: \n", which_flow_conflict)
            print("\nExisting Flow Rule Location: In switch: ",device_id, "\nConflict Details : \n",  conflict_details)
            return False, "Tie", None, None, None
        elif (winner_rule_inner == existing_rule_inner):
            print("\nExisting Flow Rule that Conflicts: \n", winner)
            print("\nThe New Flow Rule Attempted to Install based on the Given Intent: \n", non_winner)
            print("\nConflict Details : \n",  conflict_details)
            return False, "existing_rule_win", None, None, None
        else:
            print("Conflict Resolved. Winner Flow Rule: \n", winner)
            print("\nShadowed Flow Rule: \n", non_winner)
            print("\nShadowed Flow Rule Location: In switch: ",device_id, "\nConflict Details : \n",  conflict_details)
            # After resolving conflict and deciding rule1 is the winner:
            updated_flow_json = adjust_priority_ryu(winner, non_winner) #argument order important; winner first.
            intent_JSON = updated_flow_json   

    # --- 4a. Ensure a Cookie ---
    if "cookie" not in intent_JSON or intent_JSON["cookie"] == 0:
        # Generate a new, random 64-bit cookie
        intent_JSON["cookie"] = secrets.randbits(64) 
        print(f"Assigned new tracking cookie: {intent_JSON['cookie']}")
        global gl_cookie
        gl_cookie = intent_JSON['cookie']

    try:
        base_url = f"http://{RYU_CONTROLLER_IP}:{RYU_CONTROLLER_PORT}"
        success = add_flow(base_url, intent_JSON) # <-- Ryu function from [cell 5]
        if not success:
            raise Exception("add_flow returned False")
        print(f"Successfully pushed flow to dpid: {device_id}")

    except Exception as e:
            print(f"Exception found while installing flow rule: {e}")
            sys.stdout.flush()
            return False, None, None, None, None
    
    try:
        base_url = f"http://{RYU_CONTROLLER_IP}:{RYU_CONTROLLER_PORT}"
        cookie_to_check = intent_JSON.get("cookie")
        match_to_check = intent_JSON.get("match", {})
        prio_to_check = intent_JSON.get("priority", 0)

        time.sleep(0.5) # Give switch time to install
        
        # Use exists() from [cell 5]
        verification_status, operational_flow_rule = exists(
            base_url, 
            device_id, 
            match_to_check, 
            prio_to_check,
            cookie=cookie_to_check 
        )
        
        if(verification_status == True):
            # Get a unique ID for logging
            flow_id = operational_flow_rule.get("cookie", 0)
            if flow_id == 0:
                # If no cookie, fingerprint it
                flow_id = fingerprint(operational_flow_rule) # fingerprint() is from [cell 5]

            return True, flow_id, device_id, intent_JSON, operational_flow_rule
        else:
             print("Flow verification failed. Rule not found in switch.")
             return False, None, None, None, None

    except Exception as e:
            print(f"Exception found while verifying flow rule: {e}")
            sys.stdout.flush()
            return False, None, None, None, None

In [2532]:
def extract_switch_id(intent: str):
    """
    Extract the switch ID from a natural language intent.
    
    Parameters:
        intent (str): The natural language intent.
    
    Returns:
        str: Extracted switch ID (e.g., 'openflow:1') or None if not found.
    """
    # Mapping of ordinal words to numeric values
    ordinals = {
        "first": 1,
        "second": 2,
        "third": 3,
        "fourth": 4,
        "fifth": 5,
        "sixth": 6,
        "seventh": 7,
        "eighth": 8,
        "ninth": 9,
        "tenth": 10
    }

    # Match patterns like 'openflow:1'
    match = re.search(r'openflow[:\s](\d+)', intent, re.IGNORECASE)
    if match:
        return f"openflow:{match.group(1)}"

    # Match patterns like 'switch 1', 'router 2', 'node 3'
    match = re.search(r'\b(?:switch|router|node|device)(?:\s*number)?\s*(\d+)', intent, re.IGNORECASE)
    if match:
        return f"openflow:{match.group(1)}"

    # Match ordinal words (e.g., 'fourth switch', 'second router')
    match = re.search(r'\b(?:switch|router|node|device)\s*(\w+)', intent, re.IGNORECASE)
    if match:
        ordinal_word = match.group(1).lower()
        if ordinal_word in ordinals:
            return f"openflow:{ordinals[ordinal_word]}"

    # Match standalone ordinal words (e.g., 'fourth' without 'switch')
    for word, number in ordinals.items():
        if word in intent.lower():
            return f"openflow:{number}"

    return None

def execute_command(command):
    """
    Runs a command with sudo password automation.
    """
    full_command = f"echo {sudo_password} | sudo -S {command}"
    try:
        result = subprocess.run(full_command, shell=True, capture_output=True, text=True)
        if result.returncode == 0:
            return result.stdout.strip()
        else:
            raise Exception(f"Error executing command: {result.stderr.strip()}")
    except Exception as e:
        return str(e)

def get_switch_port_mapping():
    try:
        # Commands to list port and QoS configurations
        list_ports_command = "sudo -S ovs-vsctl list port"
        list_qos_command = "sudo -S ovs-vsctl list qos"
        # Fetch port and QoS data
        ports_output = execute_command(list_ports_command)
        qos_output = execute_command(list_qos_command)

        # Parse QoS data into a dictionary
        qos_mapping = {}
        current_qos = None
        for line in qos_output.splitlines():
            if line.startswith("_uuid"):
                current_qos = line.split(":")[1].strip()
            elif line.startswith("queues") and current_qos:
                qos_mapping[current_qos] = line.split(":")[1].strip()

        # Create a dictionary to store switch-to-port mapping
        switch_port_dict = {}

        # Parse ports data and check for QoS
        current_port = None
        for line in ports_output.splitlines():
            if line.startswith("name"):
                current_port = line.split(":")[1].strip()
            elif line.startswith("qos") and "[]" not in line and current_port:
                qos_uuid = line.split(":")[1].strip()

                # Extract the OpenFlow switch ID and port number
                if "-" in current_port:
                    switch, port = current_port.split("-")
                    switch_id = f"openflow:{switch[1:]}"  # e.g., "s1" -> "openflow:1"
                    port_number = port[3:]  # e.g., "eth2" -> "2"

                    # Add to dictionary
                    if switch_id not in switch_port_dict:
                        switch_port_dict[switch_id] = []
                    switch_port_dict[switch_id].append(port_number)

                current_port = None

        return switch_port_dict

    except Exception as e:
        print(f"Error: {e}")
        return {}
    
def extract_port_number(text: str):
    """
    Extract the Ethernet port number from a natural language text.
    
    Parameters:
        text (str): The input text containing the port reference.
    
    Returns:
        int: Extracted port number or None if not found.
    """
    # Mapping of ordinal words to numeric values
    ordinals = {
        "first": 1,
        "second": 2,
        "third": 3,
        "fourth": 4,
        "fifth": 5,
        "sixth": 6,
        "seventh": 7,
        "eighth": 8,
        "ninth": 9,
        "tenth": 10
    }

    # Match explicit numbers after keywords
    match = re.search(r'\b(?:port|interface|output\s+node\s+connector|ethernet)\s*(\d+)', text, re.IGNORECASE)
    if match:
        return int(match.group(1))

    # Match ordinal words (e.g., 'second port', 'third interface')
    match = re.search(r'\b(?:port|interface|output\s+node\s+connector|ethernet)\s*(\w+)', text, re.IGNORECASE)
    if match:
        ordinal_word = match.group(1).lower()
        if ordinal_word in ordinals:
            return ordinals[ordinal_word]

    # Match standalone ordinal words (e.g., 'second')
    for word, number in ordinals.items():
        if word in text.lower():
            return number

    return None

def ovs_port_exists(switch_name, port_name, sudo_password=sudo_password):
    cmd = f"sudo -S ovs-vsctl list-ports {switch_name}"
    result = subprocess.run(cmd, shell=True, capture_output=True, text=True, input=sudo_password + "\n")
    ports = result.stdout.split()
    return port_name in ports

def get_switch_name_from_device_id(dpid: int):
    # dpid is now a simple integer, e.g., 2
    return f"s{dpid}"

def create_two_queue_for_switch(device_id, port, max_rate=10000000, queue_configs=None):
    """
    Creates queues dynamically for a specific switch and port.
    
    Parameters:
        switch (str): The name of the switch in 'openflow:X' format (e.g., 'openflow:4').
        port (int): The port number on the switch (e.g., 2).
        max_rate (int): Maximum rate for the QoS (default is 10000000).
        queue_configs (list): List of tuples specifying min-rate and max-rate for each queue (default is 2 queues).
    """
    if queue_configs is None:
        # Default to 2 queues with these configurations
        queue_configs = [
            (6000000, 6000000),  # Queue 0: min-rate and max-rate
            (4000000, 4000000)   # Queue 1: min-rate and max-rate
        ]

    # Construct the port name from the input
    #port_name = f"{switch.replace('openflow:', 's')}-eth{port}"
    switch_name = get_switch_name_from_device_id(device_id)
    port_name = f"{switch_name}-eth{port}"

    if not ovs_port_exists(switch_name, port_name):
        print(f"Port {port_name} does not exist on bridge {switch_name}!")
        return


    # Construct the QoS command for the specific switch and port
    qos_command = f"sudo -S ovs-vsctl -- set port {port_name} qos=@newqos -- --id=@newqos create qos type=linux-htb other-config:max-rate={max_rate}"
    for i, (min_rate, max_rate) in enumerate(queue_configs):
        qos_command += f" queues:{i}=@q{i}"
    for i, (min_rate, max_rate) in enumerate(queue_configs):
        qos_command += f" -- --id=@q{i} create queue other-config:min-rate={min_rate} other-config:max-rate={max_rate}"

    # Execute the command
    print(f"Running: {qos_command}")
    process = subprocess.Popen(qos_command, shell=True, stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    stdout, stderr = process.communicate(input=f"{sudo_password}\n")
    if process.returncode == 0:
        print(f"Success:\n{stdout}")
    else:
        print(f"Error:\n{stderr}")


def create_two_queue_for_switch_handler(slicing_info):

    if 'use_queue' in slicing_info:         
            slicing_status = slicing_info['use_queue']
            slicing_switch_id = slicing_info['switch_id']
            slicing_queue_id = slicing_info['queue_id']
            slicing_port_id = slicing_info['port_id']

            if(slicing_status == 1):
                #openflow_id = extract_switch_id(slicing_switch_id)
                dpid = extract_dpid_for_ryu(slicing_switch_id)
                if dpid is None:
                    print("Error: Could not extract DPID for queue creation.")
                    return
                
                openflow_id_str = f"openflow:{dpid}"

                switch_port_mapping = get_switch_port_mapping()

                print("\nCheckpoint*******Entering Slice/Queue Management\n\n******")

                port_number = extract_port_number(slicing_port_id)
                
                if openflow_id_str not in switch_port_mapping :
                    
                        print("\n\nQueue was not installed in ",openflow_id, "\nInstalling now on interface: ", port_number,"\n")
                        print("\nCheckpoint*******Entering queue creation\n\n******")

                        create_two_queue_for_switch(
                            device_id=dpid,  port=port_number,
                            queue_configs=[
                                (6000000, 6000000),  # Queue 0
                                (4000000, 4000000)   # Queue 1
                            ]
                            )   
                else:
                    if str(port_number) not in switch_port_mapping[openflow_id_str]:

                        print("\n\nQueue was not installed in ",openflow_id_str, " interface: ", port_number, "\nInstalling now...\n")
                        print("\nCheckpoint*******Entering queue creation\n\n******")

                        create_two_queue_for_switch(
                            device_id=dpid,  port=port_number,
                            queue_configs=[
                                (6000000, 6000000),  # Queue 0
                                (4000000, 4000000)   # Queue 1
                            ]
                            )
    

In [2533]:
def generate_corrective_action_prompt_ryu(intent_nl, operational_flow_rule, dpid, ping_count,
                                          candidate_src_ip, candidate_dst_ip, ping_output):
    """
    Generates a prompt for the LLM to suggest corrective actions for a FAILED security intent.
    This version is adapted for Ryu's flow format.
    """
    # Get existing flows from the switch using our Ryu helper
    base_url = f"http://{RYU_CONTROLLER_IP}:{RYU_CONTROLLER_PORT}"
    existing_flows_dict = get_flows(base_url, dpid)
    existing_flows = existing_flows_dict.get(str(dpid), [])

    prompt_sections = []
    prompt_sections.append(
        "You are an SDN network assistant. Your task is to recommend a ranked list of corrective actions "
        "to enforce a **security intent** that failed during assurance testing."
    )
    prompt_sections.append(f"1. **Security Intent (in Natural Language)**:\n{intent_nl}")

    # Use Ryu's flow format
    prompt_sections.append("2. **Ryu Flow Rule for the Security Intent (Operational in Switch)**:")
    prompt_sections.append(json.dumps(operational_flow_rule, indent=2))

    prompt_sections.append("3. **Existing Flow Rules in the Same Switch (DPID)**:")
    prompt_sections.append(json.dumps(existing_flows, separators=(",", ":")))

    prompt_sections.append(
        f"""4. **Assurance Test Result**:
        - Ping Source IP: {candidate_src_ip}
        - Ping Destination IP: {candidate_dst_ip}
        - Ping Count: {ping_count}
        - Ping Output:
        {ping_output}"""
    )
    
    # Instructions updated for Ryu's JSON structure
    prompt_sections.append(
        "---\nNow, based on the above data, generate a ranked list of corrective actions. "
        "You must use only the following predefined corrective actions:\n"
        "1. Correct Match Fields\n"
        "2. Increase Priority\n"
        "3. Fix Action Field\n\n"
        "For each action:\n"
        "- **For 'Correct Match Fields'**: Specify *exactly* which key-value pairs should be set in the 'match' object (e.g., \"ipv4_src\": \"10.0.1.1\", \"eth_type\": 2048), and which keys (if any) should be removed from the 'match' object.\n"
        "- **For 'Increase Priority'**: Indicate which existing rule(s) (by cookie or match) are overshadowing the candidate rule, their current priority value(s), and the exact priority value the candidate rule should be set to.\n"
        "- **For 'Fix Action Field'**: Only include this if the 'actions' list is NOT empty ([]). The suggestion should be to set 'actions' to [].\n\n"
        "Rank the actions and explain your reasoning.\n"
        "Return your answer ONLY in the following strict JSON format:\n"
        "{\n"
        "  \"recommended_actions\": [\n"
        "    {\n"
        "      \"rank\": 1,\n"
        "      \"action\": \"Correct Match Fields\",\n"
        "      \"suggestion\": {\n"
        "        \"set_fields\": {\"ipv4_src\": \"10.0.0.1\", \"ipv4_dst\": \"10.0.0.2\"},\n"
        "        \"remove_fields\": [\"tcp_dst\"],\n"
        "        \"reasoning\": \"The match fields do not match the intended source and destination.\"\n"
        "      }\n"
        "    },\n"
        "    {\n"
        "      \"rank\": 2,\n"
        "      \"action\": \"Increase Priority\",\n"
        "      \"suggestion\": {\n"
        "        \"conflicting_rules\": [ {\"cookie\": \"123...\", \"priority\": 400} ],\n"
        "        \"recommended_priority\": 410,\n"
        "        \"reasoning\": \"The candidate rule is overshadowed by a rule with higher priority.\"\n"
        "      }\n"
        "    }\n"
        "  ]\n"
        "}\n"
        "Omit any action that is not relevant."
    )
    return "\n\n".join(prompt_sections)

In [2534]:
# --- NEW [cell 21] ---
def Run_assurance_LLM (assurance_prompt):
    """
    (This function is controller-agnostic and does not need changes.)
    Sends the prompt to the LLM and gets a JSON response.
    """
    for model in my_models_conflict_real:    
        try:
            time.sleep(0.1)             
            response = client.generate(model=model,
                options={'temperature': 0.3, 'num_ctx': 8192, 'top_p': 0.5, 'num_predict': 1024, 'num_gpu': 99},
                stream=False,
                system="",
                prompt=assurance_prompt,
                format='json'
            )
            output = response['response'].strip()
            response_json = json.loads(output)
            return response_json            
        except Exception as e:
            print(f"Exception found at assurance LLM for corrective action generation: {e}")
            sys.stdout.flush()
            continue
    return None # Return None if all models fail

In [2535]:
def execute_command_full(command, timeout=15, with_sudo=False):
    # Run the command as-is or under sudo (single layer)
    cmd = command
    if with_sudo:
        # -S: read password from stdin; -p '' suppresses the prompt text
        cmd = f"sudo -S -p '' {command}"
        cmd = f"printf '%s\n' {shlex.quote(sudo_password)} | {cmd}"
    try:
        res = subprocess.run(cmd, shell=True, capture_output=True, text=True, timeout=timeout)
        return res.stdout, res.stderr
    except subprocess.TimeoutExpired as e:
        print(f"Command timed out: {command}")
        stdout_str = e.stdout if isinstance(e.stdout, str) else (e.stdout.decode('utf-8', 'ignore') if e.stdout else "")
        stderr_str = e.stderr if isinstance(e.stderr, str) else (e.stderr.decode('utf-8', 'ignore') if e.stderr else "Timeout")
        return stdout_str, stderr_str


def execute_command_full_old(command, sudo_password=sudo_password, timeout=15):
    """
    Runs a shell command with sudo, providing the password.
    Returns TUPLE: (stdout_str, stderr_str)
    """
    cmd = f"echo {shlex.quote(sudo_password)} | sudo -S {command}"
    try:
        # text=True decodes stdout/stderr as strings on success
        result = subprocess.run(cmd, shell=True, 
                                capture_output=True, text=True, 
                                timeout=timeout)
        return result.stdout, result.stderr
    
    except subprocess.TimeoutExpired as e:
        print(f"Command timed out: {command}")
        # --- FIX ---
        # e.stdout and e.stderr are BYTES, so we must decode them to STR
        stdout_str = e.stdout.decode('utf-8', errors='ignore') if e.stdout else ""
        stderr_str = e.stderr.decode('utf-8', errors='ignore') if e.stderr else "Timeout"
        return stdout_str, stderr_str
        # --- END FIX ---
        
    except Exception as e:
        print(f"An unexpected error occurred in execute_command_full: {e}")
        return "", str(e)

def get_mininet_host_pid(src_host):
    """
    Robustly get the PID of a Mininet host process (e.g., 'h1' or 'h1onos') regardless of user.
    Looks for a process with command containing 'mininet:<src_host>'.
    """
    try:
        ps_output = subprocess.check_output(["ps", "-eo", "pid,args"], text=True).strip().splitlines()
    except subprocess.CalledProcessError as e:
        raise RuntimeError("Failed to run 'ps -eo pid,args'") from e

    for line in ps_output:
        if f"mininet:{src_host}" in line and "grep" not in line:
            parts = line.strip().split(None, 1)
            if len(parts) == 2:
                pid_str, cmd = parts
                try:
                    pid = int(pid_str)
                    #print(f"[DEBUG] Matched host '{src_host}' → PID {pid}")
                    return pid
                except ValueError:
                    continue

    raise RuntimeError(f"No Mininet host process found for '{src_host}'.")

# --- NEW [cell 22] ---

def extract_host_and_ip_ryu(flow_data: dict):
    """
    Extracts source and destination IPs from a Ryu flow rule.
    Returns: (src_host, dst_host, src_ip, dst_ip)
    """
    src_ip = dst_ip = None
    match = flow_data.get("match", {})

    # Extract IPs, stripping any /mask
    if "ipv4_src" in match:
        src_ip = match["ipv4_src"].split("/")[0]
    if "ipv4_dst" in match:
        dst_ip = match["ipv4_dst"].split("/")[0]

    # Use the ip_to_host map from [cell 3]
    src_host = ip_to_host.get(src_ip) if src_ip else None
    dst_host = ip_to_host.get(dst_ip) if dst_ip else None

    return src_host, dst_host, src_ip, dst_ip

#def ryu_assurance_for_security_intent(src_ip, dst_ip, ping_count=2):
def ryu_assurance_for_security_intent(src_ip, dst_ip, rule_dpid, ping_count=2):
    """
    (This function is mostly controller-agnostic, as it tests the data plane.)
    (The 'get_mininet_host_pid' and 'execute_command_full' helpers are assumed to be in [cell 21] or [cell 22])
    """
    all_ips = ["10.0.1.1", "10.0.1.2", "10.0.1.3", "10.0.1.4"]
    global llm_caller_flag
    llm_caller_flag = 0

    def perform_ping(source_ip, target_ip):
        source_host = ip_to_host.get(source_ip)
        if not source_host:
            print(f"[WARNING] Unknown source host for IP: {source_ip}")
            return ""
        try:
            host_pid = get_mininet_host_pid(source_host)
        except Exception as e:
            print(f"[ERROR] Cannot get PID for host {source_host}: {e}")
            return ""
        ping_cmd = f"mnexec -a {host_pid} ping -c {ping_count} {target_ip}"
        out, err = execute_command_full(ping_cmd, with_sudo=True)
        return f"{out or ''}{err or ''}"

    # def perform_ping(source_ip, target_ip):
    #     source_host = ip_to_host.get(source_ip)
    #     if not source_host:
    #         print(f"[WARNING] Unknown source host for IP: {source_ip}")
    #         return None
    #     try:
    #         host_pid = get_mininet_host_pid(source_host)
    #     except Exception as e:
    #         print(f"[ERROR] Cannot get PID for host {source_host}: {e}")
    #         return None
    #     ping_cmd = f"echo {sudo_password} | sudo -S mnexec -a {host_pid} ping -c {ping_count} {target_ip}"
    #     #return execute_command_full(ping_cmd, timeout=15)
    #     return execute_command_full(ping_cmd, with_sudo=True)


    # (The rest of the ping logic from your original cell 22 is correct)
    # Case 1: src and dst IP both present
    if src_ip and dst_ip:
        print(f"[INFO] Testing {src_ip} to {dst_ip}")
        output = perform_ping(src_ip, dst_ip)
        if output and "100% packet loss" in output:
            print(f"[PASS] Intent effective. Traffic from {src_ip} to {dst_ip} is blocked.")
        elif output and "0% packet loss" in output:
            print(f"[FAIL] Intent NOT effective. Traffic from {src_ip} to {dst_ip} is NOT blocked.")
            llm_caller_flag = 1
        else:
            print("[WARN] Inconclusive result. Ping output:\n", output, "\nPlease check first why ping is not working.")
            llm_caller_flag = 2
        return ping_count, src_ip, dst_ip, output
    
    # Case 1: src and dst IP both present
    if src_ip and dst_ip:
        print(f"[INFO] Testing {src_ip} to {dst_ip}")
        output = perform_ping(src_ip, dst_ip)
        if output and "100% packet loss" in output:
            print(f"[PASS] Intent effective. Traffic from {src_ip} to {dst_ip} is blocked.")
        elif output and "0% packet loss" in output:
            print(f"[FAIL] Intent NOT effective. Traffic from {src_ip} to {dst_ip} is NOT blocked.")
            llm_caller_flag = 1
        else:
            print("[WARN] Inconclusive result. Ping output:\n", output, "\nPlease check first why ping is not working.")
            llm_caller_flag = 2
        return ping_count, src_ip, dst_ip, output

    # Case 2: src_ip is None → test all src IPs → dst_ip
    elif src_ip is None and dst_ip:
        print(f"[INFO] Testing multiple sources to {dst_ip}")
        for candidate_src in all_ips:
            if candidate_src == dst_ip:
                continue
            output = perform_ping(candidate_src, dst_ip)
            if output and "100% packet loss" in output:
                continue  # this one is blocked, good
            elif output and "0% packet loss" in output:
                print(f"[FAIL] Intent NOT effective. {candidate_src} → {dst_ip} was NOT blocked.")
                llm_caller_flag = 1
                return ping_count, candidate_src, dst_ip, output
            else:
                print(f"[WARN] Inconclusive result from {candidate_src} → {dst_ip}:\n{output}", "\nPlease check first why ping is not working.")
                llm_caller_flag = 2
                return ping_count, candidate_src, dst_ip, output
        print(f"[PASS] Intent effective. All sources blocked from reaching {dst_ip}.")
        return ping_count, candidate_src, dst_ip, output

    # Case 3: dst_ip is None → test src_ip → all destinations
    elif dst_ip is None and src_ip:
        print(f"[INFO] Testing {src_ip} for multiple destinations")
        for candidate_dst in all_ips:
            if candidate_dst == src_ip:
                continue

            # --- START NEW LOGIC ---
            # (Make sure HOST_ATTACH and SW_OF from cell 2023 are accessible)
            try:
                src_attach_dev_str, _ = HOST_ATTACH.get(src_ip, (None, None))
                dst_attach_dev_str, _ = HOST_ATTACH.get(candidate_dst, (None, None))
                
                # Convert the integer rule_dpid (e.g., 2) to its 'of:...' string
                rule_dev_str = SW_OF.get(str(rule_dpid))

                # If hosts are on the same switch, AND that switch is NOT where the rule is,
                # this path is irrelevant. Skip the test.
                if (src_attach_dev_str and 
                    src_attach_dev_str == dst_attach_dev_str and 
                    src_attach_dev_str != rule_dev_str):
                    
                    print(f"[INFO] Skipping test {src_ip} -> {candidate_dst} (local on {src_attach_dev_str}, rule is on {rule_dev_str})")
                    continue
            except Exception as e:
                print(f"[WARN] Topology check failed: {e}. Proceeding with test.")
            # --- END NEW LOGIC ---

            output = perform_ping(src_ip, candidate_dst)
            if output and "100% packet loss" in output:
                continue  # this one is blocked, good
            elif output and "0% packet loss" in output:
                print(f"[FAIL] Intent NOT effective. {src_ip} → {candidate_dst} was NOT blocked.")
                llm_caller_flag = 1
                return ping_count, src_ip, candidate_dst, output
            else:
                print(f"[WARN] Inconclusive result from {src_ip} → {candidate_dst}:\n{output}", "\nPlease check first why ping is not working.")
                llm_caller_flag = 2
                return ping_count, src_ip, candidate_dst, output
        print(f"[PASS] Intent effective.")
        return ping_count, src_ip, candidate_dst, output
    
    else:
        print("[ERROR] Both source and destination IPs are missing. Cannot evaluate intent.")
        llm_caller_flag = 2
        return ping_count, None, None, None


def correct_match_fields_ryu(candidate_flow, set_fields, remove_fields):
    """
    Update candidate_flow's match fields for Ryu.
    set_fields: dict, e.g. {'ipv4_src': '10.0.2.2'}
    remove_fields: list of keys, e.g. ['tcp_dst']
    """
    match = candidate_flow.get("match", {})
    
    # Remove fields
    for field in remove_fields:
        match.pop(field, None) # Safely remove
    
    # Set/update fields
    match.update(set_fields)
    
    candidate_flow['match'] = match
    return candidate_flow

def increase_priority_ryu(candidate_flow, recommended_priority):
    candidate_flow['priority'] = recommended_priority
    return candidate_flow

def fix_action_field_ryu(candidate_flow):
    """
    Make sure actions list is empty (for a block rule).
    """
    candidate_flow['actions'] = []
    return candidate_flow

def parse_and_execute_corrective_actions_ryu(candidate_flow, llm_response, dpid):
    """
    Applies corrective actions suggested by the LLM, one by one, and re-verifies.
    'candidate_flow' is the Ryu flow rule that failed verification.
    """
    if not llm_response or "recommended_actions" not in llm_response:
        print("LLM provided no valid corrective actions.")
        return False
        
    actions = sorted(llm_response["recommended_actions"], key=lambda x: x["rank"])
    base_url = f"http://{RYU_CONTROLLER_IP}:{RYU_CONTROLLER_PORT}"
    
    # We need the original src/dst for re-verification
    _, _, src_ip, dst_ip = extract_host_and_ip_ryu(candidate_flow)

    # We must delete the old (bad) flow before adding the corrected one
    print("Deleting the original, failed flow rule...")
    delete_flow(base_url, candidate_flow)
    
    original_cookie = candidate_flow.get("cookie", 0) # Keep track of original cookie
    
    for action_item in actions:
        action = action_item["action"]
        suggestion = action_item.get("suggestion", {})
        print(f"Triggering action: {action} (rank {action_item['rank']})")

        # Make a copy to modify
        corrected_flow = candidate_flow.copy()
        
        if action == "Correct Match Fields":
            set_fields = suggestion.get("set_fields", {})
            remove_fields = suggestion.get("remove_fields", [])
            corrected_flow = correct_match_fields_ryu(corrected_flow, set_fields, remove_fields)
        
        elif action == "Increase Priority":
            recommended_priority = suggestion.get("recommended_priority")
            if recommended_priority:
                corrected_flow = increase_priority_ryu(corrected_flow, recommended_priority)
            else:
                print("Warning: LLM suggested priority increase but gave no value.")
                continue
                
        elif action == "Fix Action Field":
            corrected_flow = fix_action_field_ryu(corrected_flow)
        
        else:
            print(f"Unknown action: {action}")
            continue

        # Ensure the cookie remains the same for tracking
        corrected_flow["cookie"] = original_cookie
        
        # --- 2. Push corrected flow ---
        print(f"Pushing corrected flow: {json.dumps(corrected_flow)}")
        try:
            add_flow(base_url, corrected_flow)
            time.sleep(0.5) # Settle time
        except Exception as e:
            print(f"Exception while pushing corrected flow: {e}")
            continue # Try next action

        # --- 3. Re-verify ---
        print("Re-verifying intent...")
        global llm_caller_flag
        # This will ping and set llm_caller_flag (0=PASS, 1=FAIL)
        ryu_assurance_for_security_intent(src_ip, dst_ip) 
        
        if llm_caller_flag == 0:
            print(f"Intent deviation resolved after action: {action}")
            return True  # Deviation fixed
        else:
            print(f"Deviation not fixed after action: {action}. Deleting flow and trying next action...")
            delete_flow(base_url, corrected_flow) # Clean up before next loop

    print("\nAll suggested corrective actions exhausted, but deviation remains. Escalate to operator.")
    return False  # Deviation not fixed

def ONOS_assurance_for_qos_intent(src_host, dst_host, src_ip, dst_ip, ping_count=2):

    print("")

def ONOS_assurance_for_forwarding_intent(src_host, dst_host, src_ip, dst_ip, ping_count=2):

    print("")

In [2536]:
def correct_match_fields(candidate_flow, set_fields, remove_fields):
    """
    Update candidate_flow's match fields: set specified, remove specified.
    set_fields: dict, e.g. {'IPV4_SRC': '10.0.2.2/32'}
    remove_fields: list of type strings
    """
    selector = candidate_flow['selector']
    # Remove fields
    selector['criteria'] = [
        crit for crit in selector['criteria'] if crit['type'] not in remove_fields
    ]
    # Set/update fields
    for k, v in set_fields.items():
        # Remove any existing
        selector['criteria'] = [crit for crit in selector['criteria'] if crit['type'] != k]
        # Correct value key based on ONOS spec
        if k in ["IPV4_SRC", "IPV4_DST"]:
            selector['criteria'].append({'type': k, 'ip': v})
        elif k == "ETH_TYPE":
            selector['criteria'].append({'type': k, 'ethType': v})
        else:
            selector['criteria'].append({'type': k, 'value': v})
    candidate_flow['selector'] = selector
    return candidate_flow

def increase_priority(candidate_flow, recommended_priority):
    candidate_flow['priority'] = recommended_priority
    return candidate_flow

def fix_action_field(candidate_flow):
    """
    Make sure treatment.instructions is only [{'type': 'NOACTION'}]
    """
    candidate_flow['treatment'] = {
        "instructions": [{"type": "NOACTION"}]
    }
    return candidate_flow

def parse_and_execute_corrective_actions_onos(candidate_flow, llm_response, device_id, flow_id, src_ip, dst_ip):
    """
    candidate_flow: dict, the ONOS flow rule you want to fix
    existing_flows: list of dicts, flows on the same device
    llm_response: dict, output from LLM with 'recommended_actions'
    intent_drift_fn: function that checks for intent drift, returns True if drift still exists, False if resolved
    push_flow_fn: function to push the modified flow to ONOS
    """
    actions = sorted(llm_response["recommended_actions"], key=lambda x: x["rank"])
    for action_item in actions:
        action = action_item["action"]
        suggestion = action_item["suggestion"]
        print(f"Triggering action: {action} (rank {action_item['rank']})")

        if action == "Correct Match Fields":
            set_fields = suggestion.get("set_fields", {})
            remove_fields = suggestion.get("remove_fields", [])
            candidate_flow = correct_match_fields(candidate_flow, set_fields, remove_fields)
        elif action == "Increase Priority":
            recommended_priority = suggestion.get("recommended_priority")
            candidate_flow = increase_priority(candidate_flow, recommended_priority)
        elif action == "Fix Action Field":
            candidate_flow = fix_action_field(candidate_flow)
        else:
            print(f"Unknown action: {action}")

        # Push the flow to ONOS
        # First remove the old one
        delete_flow_rule_ONOS(device_id, flow_id)

        try:
            flow_id = push_flow_rule(device_id, candidate_flow)
        except Exception as e:
                print("Exception found while installing flow rule: ", e)
                sys.stdout.flush()
        try:
            verification_status, operational_flow_rule = verify_flow_rule(device_id, flow_id)
            if(verification_status == True):
                print("\nCorrected Flow Rules Installed Successfully\n")
        except Exception as e:
                print("Corrected Flow Rules Failed to be Installed Successfully. Exception found while verifying flow rule: ", e)
                sys.stdout.flush()

        global llm_caller_flag

        ryu_assurance_for_security_intent(src_ip, dst_ip)

        # Check intent drift (assurance) after each action
        
        if llm_caller_flag == 0:
            print(f"Intent deviation resolved after action: {action}")
            return True  # Deviation fixed

        print(f"Deviation not fixed after action: {action}, proceeding to next action...")

    print("\n\nAll suggested corrective actions exhausted, but deviation remains. Escalate the issue to the Operator.\n")
    return False  # Deviation not fixed


In [2537]:
#QoS verification helper functions

_UNITS_BYTES = {"KBytes":1024, "MBytes":1024**2, "GBytes":1024**3, "TBytes":1024**4, "Bytes":1}
_UNITS_BITS  = {"Kbits/sec":1e3, "Mbits/sec":1e6, "Gbits/sec":1e9, "bits/sec":1.0}

def build_pin_selector_forward(src_ip: str, dst_ip: str, dport: int, proto: str = "tcp"):
    proto = proto.lower()
    criteria = [
        {"type": "ETH_TYPE", "ethType": "0x800"},
        {"type": "IP_PROTO", "protocol": 6 if proto == "tcp" else 17},
        {"type": "IPV4_SRC", "ip": f"{src_ip}/32"},
        {"type": "IPV4_DST", "ip": f"{dst_ip}/32"},
    ]
    if proto == "tcp":
        criteria.append({"type": "TCP_DST", "tcpPort": dport})
    else:
        criteria.append({"type": "UDP_DST", "udpPort": dport})
    return {"criteria": criteria}

def build_pin_selector_reverse(src_ip: str, dst_ip: str, sport: int, proto: str = "tcp"):
    # reverse direction (dst->src); for TCP we match TCP_SRC=port, for UDP we match UDP_SRC=port
    proto = proto.lower()
    criteria = [
        {"type": "ETH_TYPE", "ethType": "0x800"},
        {"type": "IP_PROTO", "protocol": 6 if proto == "tcp" else 17},
        {"type": "IPV4_SRC", "ip": f"{dst_ip}/32"},
        {"type": "IPV4_DST", "ip": f"{src_ip}/32"},
    ]
    if proto == "tcp":
        criteria.append({"type": "TCP_SRC", "tcpPort": sport})
    else:
        criteria.append({"type": "UDP_SRC", "udpPort": sport})
    return {"criteria": criteria}

def build_output_only_treatment(out_port: int | str):
    return {"instructions": [{"type": "OUTPUT", "port": str(out_port)}]}

def build_queue_and_output_treatment(queue_id: int, out_port: int | str):
    return {"instructions": [{"type": "QUEUE", "queueId": int(queue_id)}, {"type": "OUTPUT", "port": str(out_port)}]}

def build_pin_flow_body(device_id: str, selector: dict, treatment: dict, priority: int = 65000):
    # ONOS expects an array under "flows"
    return {
        "flows": [{
            "deviceId": device_id,
            "isPermanent": True,
            "priority": priority,
            "selector": selector,
            "treatment": treatment
        }]
    }

# def delete_flow_rule(device_id: str, flow_id: str):
#     url = f"{ONOS_BASE_URL}/{device_id}/{flow_id}"
#     HEADERS = {"Accept": "application/json"}
#     r = requests.delete(url, headers=HEADERS, auth=(USERNAME, PASSWORD), timeout=15)
#     # 204/200/202 = fine; 404 -> already gone is OK
#     if r.status_code not in (200, 202, 204, 404):
#         print(f"[WARN] DELETE {device_id}/{flow_id} failed {r.status_code}: {r.text}")

# ---------- PIN PATH VIA S3 (place below the helpers) ----------
def pin_path_via_s3(src_ip="10.0.1.1", dst_ip="10.0.1.3", dst_port=80, proto="tcp",
                    include_s4_forward_qos=False, s4_queue_id=1):
    """
    Installs high-priority OUTPUT-only pins on s1 and s3 for forward,
    OUTPUT-only pins on s4,s3,s1 for reverse ACK path.
    Optionally installs the s4 forward QoS rule (QUEUE+OUTPUT) if needed.
    Returns dict of {label: (deviceId, flowId)} for cleanup.
    """
    S1, S3, S4 = "of:0000000000000001", "of:0000000000000003", "of:0000000000000004"
    pins = {}

    # Selectors
    sel_fwd = build_pin_selector_forward(src_ip, dst_ip, dport=dst_port, proto=proto)
    sel_rev = build_pin_selector_reverse(src_ip, dst_ip, sport=dst_port, proto=proto)

    # ---- Forward pins (s1->s3->s4). s4 forward QoS rule optional. ----
    # s1 out to s3 (port 2)
    body = build_pin_flow_body(S1, sel_fwd, build_output_only_treatment(2), priority=65000)
    fid = push_flow_rule(S1, body); pins["s1_fwd"] = (S1, fid)

    # s3 out to s4 (port 2)
    body = build_pin_flow_body(S3, sel_fwd, build_output_only_treatment(2), priority=65000)
    fid = push_flow_rule(S3, body); pins["s3_fwd"] = (S3, fid)

    if include_s4_forward_qos:
        # s4 set-queue then out to h3 (port 3)
        body = build_pin_flow_body(S4, sel_fwd, build_queue_and_output_treatment(s4_queue_id, 3), priority=65000)
        fid = push_flow_rule(S4, body); pins["s4_qos_fwd"] = (S4, fid)

    # ---- Reverse (ACK) pins (h3->h1) along s4->s3->s1 ----
    # s4 out to s3 (port 2)
    body = build_pin_flow_body(S4, sel_rev, build_output_only_treatment(2), priority=65000)
    fid = push_flow_rule(S4, body); pins["s4_rev"] = (S4, fid)

    # s3 out to s1 (port 1)
    body = build_pin_flow_body(S3, sel_rev, build_output_only_treatment(1), priority=65000)
    fid = push_flow_rule(S3, body); pins["s3_rev"] = (S3, fid)

    # s1 out to h1 (port 3)
    body = build_pin_flow_body(S1, sel_rev, build_output_only_treatment(3), priority=65000)
    fid = push_flow_rule(S1, body); pins["s1_rev"] = (S1, fid)

    return pins

def run_in_host(host_pid: int, cmd: str, timeout: int = 30) -> tuple[str, str]:
    # Enter the host namespace; sudo is applied by execute_command_full
    base = f"mnexec -a {host_pid} {cmd}"
    return execute_command_full(base, timeout=timeout, with_sudo=True)

def run_in_host_old(host_pid: int, cmd: str, timeout: int = 30, require_sudo: bool = True) -> str:
    """
    Run a command inside a Mininet host namespace using mnexec -a <PID>.
    Always uses sudo here to avoid 'Permission denied' when entering namespaces.
    """
    #prefix = f"echo {shlex.quote(sudo_password)} | sudo -S "
    #full_cmd = f"{prefix}mnexec -a {host_pid} {cmd}"

    full_cmd = f"mnexec -a {host_pid} {cmd}"
    #return execute_command_full(full_cmd, timeout=timeout)
    return execute_command_full(full_cmd, with_sudo=True)

def ensure_no_iperf_server_old2(host_pid: int, port: int) -> None:
    # Fast attempt
    run_in_host(host_pid, f"pkill -f {shlex.quote(f'iperf3 -s -p {int(port)}')} || true", timeout=5)
    # Fallback: precise kill (note doubled braces in awk)
    run_in_host(
        host_pid,
        f"bash -lc \"pgrep -af 'iperf3.*-s.*-p {int(port)}' | awk '{{{{print $1}}}}' | xargs -r kill -9\"",
        timeout=5
    )

def ensure_no_iperf_server(host_pid: int, port: int) -> None:
    # Fast attempt
    run_in_host(host_pid, f"/usr/bin/pkill -f {shlex.quote(f'iperf3 -s -p {int(port)}')} || true", timeout=5)
    # Fallback: precise kill (note doubled braces in awk)
    run_in_host(
        host_pid,
        # FIX: Changed \\" to \" to correctly escape the quotes
        f"bash -lc \"/usr/bin/pgrep -af 'iperf3.*-s.*-p {int(port)}' | /usr/bin/awk '{{{{print $1}}}}' | /usr/bin/xargs -r /bin/kill -9\"",
        timeout=5
    )


def ensure_no_iperf_server_old(host_pid: int, port: int) -> None:
    """
    Attempt to kill any existing iperf3 server on that port in the host namespace.
    """
    # Try pkill first, then kill by pgrep if needed.
    run_in_host(host_pid, f"pkill -f {shlex.quote(f'iperf3 -s -p {port}')} || true", timeout=5, require_sudo=True)
    run_in_host(host_pid, f"bash -lc \"pgrep -af 'iperf3.*-s.*-p {port}' | awk '{{print $1}}' | xargs -r kill -9\"", timeout=5, require_sudo=True)


def start_iperf_server(host_pid: int, port: int, extra_args: str = "") -> None:
    ensure_no_iperf_server(host_pid, port)

    run_in_host(
            host_pid,
            f"bash -lc 'nohup /usr/bin/iperf3 -s -p {int(port)} --one-off {extra_args} "
            f">/tmp/iperf3_s_{int(port)}.log 2>&1 & echo $!'",
            timeout=5
        )
    
    deadline = time.time() + 4.0
    while time.time() < deadline:
        out, err = run_in_host(
            host_pid,
            f"bash -lc \"/usr/bin/ss -ltnp | grep ':{int(port)} ' || true\"",
            timeout=3
        )
        if out.strip():
            return  # Server is listening on port

    # Not listening: surface the server log to explain why
    log, _ = run_in_host(
        host_pid,
        f"bash -lc 'tail -n +200 /tmp/iperf3_s_{int(port)}.log 2>/dev/null || true'",
        timeout=3
    )
    raise RuntimeError(f"iperf3 server failed to bind/listen on port {port}. Server log:\n{log}")


def start_iperf_server_old2(host_pid: int, port: int, extra_args: str = "") -> None:
    ensure_no_iperf_server(host_pid, port)

    run_in_host(
            host_pid,
            f"bash -lc 'nohup iperf3 -s -p {int(port)} --one-off {extra_args} "
            f">/tmp/iperf3_s_{int(port)}.log 2>&1 & echo $!'",
            timeout=5
        )
    
    deadline = time.time() + 4.0
    while time.time() < deadline:
        out, err = run_in_host(
            host_pid,
            f"bash -lc \"ss -ltnp | grep ':{int(port)} ' || true\"",
            timeout=3
        )
        if out.strip():
            return  # Server is listening on port
        time.sleep(0.2)

    # Not listening: surface the server log to explain why
    log, _ = run_in_host(
        host_pid,
        f"bash -lc 'tail -n +200 /tmp/iperf3_s_{int(port)}.log 2>/dev/null || true'",
        timeout=3
    )
    raise RuntimeError(f"iperf3 server failed to bind/listen on port {port}. Server log:\n{log}")

    #cmd = f"bash -lc 'nohup iperf3 -s -p {int(port)} --one-off {extra_args} >/tmp/iperf3_s_{int(port)}.log 2>&1 & echo $!'"
    #run_in_host(host_pid, cmd, timeout=5)
    #time.sleep(0.6)  # small readiness delay


def start_iperf_server_old(host_pid: int, port: int, extra_args: str = "") -> None:
    """
    Start iperf3 server (-s) on port (may require sudo for <1024). Run in background.
    Use --one-off to auto-exit after one test.
    """
    ensure_no_iperf_server(host_pid, port)
    cmd = f"bash -lc 'nohup iperf3 -s -p {int(port)} --one-off {extra_args} >/tmp/iperf3_s_{port}.log 2>&1 & echo $!'"
    out = run_in_host(host_pid, cmd, timeout=5, require_sudo=True)
    # Best-effort small delay for readiness
    time.sleep(0.6)

def stop_iperf_server(host_pid: int, port: int) -> None:
    ensure_no_iperf_server(host_pid, port)

def sudo_sh(cmd, timeout=20):
    # Pass ONLY the command. execute_command_full will add the sudo prefix.
    return execute_command_full(f"bash -lc {shlex.quote(cmd)}", timeout=timeout, with_sudo=True)

def sudo_sh_old(cmd, timeout=20):
    return execute_command_full(f"echo {shlex.quote(sudo_password)} | sudo -S bash -lc {shlex.quote(cmd)}", timeout=timeout, with_sudo=True)

def find_bridge_for_device(device_id: str) -> str:
    js = sudo_sh("ovs-vsctl -f json list Bridge", timeout=10)
    data = json.loads(js)
    dpid_hex = device_id.replace("of:","").lower()
    head = data["headings"]
    for row in data["data"]:
        obj = dict(zip(head, row))
        oc = dict(obj["other_config"][1]) if isinstance(obj.get("other_config"), list) else {}
        oc_dpid = (oc.get("datapath-id") or oc.get("datapath_id") or "").lower().replace(":","")
        if oc_dpid == dpid_hex:
            name = obj["name"]
            return name[-1] if isinstance(name, list) else name
    raise RuntimeError("Bridge not found")

def parse_qos_show(iface: str) -> dict:
    """
    Runs `ovs-appctl qos/show` and parses its output into a dict.
    This version correctly uses the execute_command_full helper.
    """
    # (Assumes 'sudo_password' is a global variable)
    cmd = f"ovs-appctl -t ovs-vswitchd qos/show {shlex.quote(iface)}"
    
    # Call the helper that returns a (stdout, stderr) tuple
    #stdout_text, stderr_text = execute_command_full(cmd, sudo_password, with_sudo=True)
    stdout_text, stderr_text = execute_command_full(cmd, with_sudo=True)

    if "No QoS configured" in stdout_text or "No QoS configured" in stderr_text:
        return {"raw": stdout_text, "queues": {}}
    if not stdout_text and "Timeout" in stderr_text:
        print(f"[WARN] Command timed out: {cmd}")
        return {"raw": "Timeout", "queues": {}}
    
    # --- This is the new parsing logic ---
    raw = stdout_text
    queues = {}
    qos_meta = {"raw": raw}
    current_obj = None # Can be 'qos_meta' or a specific queue dict
    
    for raw_line in raw.splitlines():
        line = raw_line.strip()
        if not line:
            continue
            
        # Check for a new section header (e.g., "QoS:", "Queue 0:", "Default:")
        header_match = re.match(r"^(QoS|Queue|Default):\s*(.*)", line)
        if header_match:
            obj_type = header_match.group(1)
            if obj_type == "QoS":
                current_obj = qos_meta
            elif obj_type == "Queue":
                q_id = line.split()[1].replace(":", "") # Get "0" from "Queue 0:"
                queues[q_id] = {}
                current_obj = queues[q_id]
            elif obj_type == "Default":
                queues["default"] = {}
                current_obj = queues["default"]
            continue
        
        # Parse key-value pairs
        if current_obj is not None:
            kv_match = re.match(r"^([\w-]+):\s*(\d+)", line)
            if kv_match:
                key = kv_match.group(1)
                try:
                    value = int(kv_match.group(2))
                except ValueError:
                    value = kv_match.group(2)
                current_obj[key] = value

    qos_meta["queues"] = queues
    return qos_meta

def parse_qos_show_old(iface: str):
    """
    Parse `ovs-appctl -t ovs-vswitchd qos/show IFACE` that looks like:

      QoS: s2onos-eth2 linux-htb
      max-rate: 10000000

      Default:
        burst: 12512
        min-rate: 6000000
        tx_packets: 1868919
        tx_bytes: 267187192
        tx_errors: 0

      Queue 1:
        burst: 12512
        min-rate: 4000000
        tx_packets: 3429
        tx_bytes: 5078775
        tx_errors: 0

    Returns: {"queues": {"default": {...}, "1": {...}}, "raw": <text>, "_qos": {"max-rate": ...}}
    """
    txt = sudo_sh(f"ovs-appctl -t ovs-vswitchd qos/show {shlex.quote(iface)} 2>&1", timeout=8)

    queues = {}
    current = None
    qos_meta = {}

    for raw in txt.splitlines():
        line = raw.strip()

        # Top-level QoS meta (e.g., max-rate)
        m_qos_max = re.search(r'\bmax-rate:\s*(\d+)', line, re.IGNORECASE)
        if m_qos_max and current is None:
            qos_meta["max-rate"] = int(m_qos_max.group(1))

        # Block headers
        if re.match(r'^(Default)\s*:\s*$', line, re.IGNORECASE):
            current = "default"
            queues.setdefault(current, {})
            continue
        m_q = re.match(r'^(?:Queue|queue)\s+(\d+)\s*:\s*$', line)
        if m_q:
            current = m_q.group(1)
            queues.setdefault(current, {})
            continue

        # Inside a block, pick out key:value lines we care about
        if current:
            for key, pat in [
                ("min-rate",   r'\bmin-rate:\s*(\d+)'),
                ("max-rate",   r'\bmax-rate:\s*(\d+)'),
                ("tx_bytes",   r'\btx_bytes:\s*(\d+)'),
                ("tx_packets", r'\btx_packets:\s*(\d+)'),
            ]:
                m = re.search(pat, line, re.IGNORECASE)
                if m:
                    queues[current][key] = int(m.group(1))

    return {"queues": queues, "raw": txt, "_qos": qos_meta}

# # keep snapshot_queue using the patched parser
def snapshot_queue(iface: str, queue_id: int | str):
    parsed = parse_qos_show(iface)
    q = parsed["queues"].get(str(queue_id)) or parsed["queues"].get("default") or {}
    return {
        "min_rate": q.get("min-rate"),
        "max_rate": q.get("max-rate"),
        "tx_bytes": q.get("tx_bytes"),
        "tx_packets": q.get("tx_packets"),
        "_raw": parsed["raw"][:1200]
    }

def _sudo(cmd, timeout=30):
    return execute_command_full(f"bash -lc {shlex.quote(cmd)}", timeout=timeout, with_sudo=True)
    #return execute_command_full(f"echo {shlex.quote(sudo_password)} | sudo -S bash -lc {shlex.quote(cmd)}", timeout=timeout)

def _ns(pid, cmd, timeout=90):
    return execute_command_full(f"mnexec -a {pid} bash -lc {shlex.quote(cmd)}", timeout=timeout, with_sudo=True)
    #return execute_command_full(f"echo {shlex.quote(sudo_password)} | sudo -S mnexec -a {pid} bash -lc {shlex.quote(cmd)}", timeout=timeout)

def _parse_iperf_sender(line: str) -> tuple[float, int] | tuple[None, None]:
    """
    Parses a single iperf sender summary line.
    Line format: [ 3] 0.0-10.0 sec  112 MBytes  93.8 Mbits/sec
    Line format: [SUM] 0.0-10.0 sec  112 MBytes  93.8 Mbits/sec
    """
    try:
        # Regex to find the transfer and bandwidth
        match = re.search(r"(\d+(?:\.\d+)?)\s+MBytes\s+(\d+(?:\.\d+)?)\s+Mbits/sec", line)
        if match:
            bytes_sent = int(float(match.group(1)) * 1024 * 1024)
            mbps = float(match.group(2))
            return mbps, bytes_sent
            
        # Fallback for Kbits
        match = re.search(r"(\d+(?:\.\d+)?)\s+KBytes\s+(\d+(?:\.\d+)?)\s+Kbits/sec", line)
        if match:
            bytes_sent = int(float(match.group(1)) * 1024)
            mbps = float(match.group(2)) / 1024.0
            return mbps, bytes_sent
            
        # Fallback for bits
        match = re.search(r"(\d+(?:\.\d+)?)\s+Bytes\s+(\d+(?:\.\d+)?)\s+bits/sec", line)
        if match:
            bytes_sent = int(float(match.group(1)))
            mbps = float(match.group(2)) / 1_000_000.0
            return mbps, bytes_sent

    except Exception as e:
        print(f"[WARN] Failed to parse iperf line '{line}': {e}")
        pass
    return None, None

def _iperf_text_summary(client_pid, dst_ip, dst_port, duration, extra_args="") -> dict:
    """
    Runs iperf and returns the parsed summary text.
    This version correctly finds the [SUM] line for parallel tests.
    """
    # Build the command *without* the password
    #cmd = f"mnexec -a {client_pid} iperf -c {dst_ip} -p {dst_port} -t {duration} {extra_args}"
    #cmd = f"mnexec -a {client_pid} iperf3 -c {dst_ip} -p {dst_port} -t {duration} {extra_args}" # <-- FIX
    iperf_cmd_str = f'/usr/bin/iperf3 -c {dst_ip} -p {dst_port} -t {duration} {extra_args}'
    cmd = f"mnexec -a {client_pid} bash -lc {shlex.quote(iperf_cmd_str)}"

    #cmd = f"mnexec -a {client_pid} iperf3 -c {dst_ip} -p {dst_port} -t {duration} {extra_args}"
    # stdout_text, stderr_text = execute_command_full(cmd, timeout=max(15, duration + 10), with_sudo=True)

    
    # # Call the new helper, which handles sudo and returns two values
    # # (Assumes 'sudo_password' is a global variable)
    # stdout_text, stderr_text = execute_command_full(cmd, sudo_password, timeout=max(15, duration + 10), with_sudo=True)

    stdout_text, stderr_text = execute_command_full(
        cmd,
        timeout=max(15, duration + 10),
        with_sudo=True
    )
    
    if "Connection refused" in stderr_text:
        raise RuntimeError(f"Iperf connection refused. Is server running on {dst_ip}:{dst_port}?")
    if "No route to host" in stderr_text:
        raise RuntimeError(f"Iperf 'No route to host' for {dst_ip}. Check pinning flows.")

    stdout_lines = stdout_text.splitlines()
    
    target_line = None
    for ln in reversed(stdout_lines): # Search from the bottom up
        if ("bits/sec" in ln and 
           (f" 0.0-{duration:.1f}" in ln or f" 0.00-{duration:.02f}" in ln or " 0.0-" in ln)):
            
            if ln.strip().startswith("[SUM]"):
                target_line = ln # Prefer SUM line
                break
            if ln.strip().startswith("["):
                target_line = ln # Fallback to any summary line
    
    if not target_line:
        print("[DEBUG] iperf stdout:", stdout_text)
        print("[DEBUG] iperf stderr:", stderr_text)
        raise RuntimeError("Could not find iperf sender summary line. iperf output was not as expected.")

    mbps, bytes_sent = _parse_iperf_sender(target_line)
    return {"sender_mbps": mbps, "bytes_sent": bytes_sent, "sender_line": target_line}

def _parse_iperf_sender_old(line: str):
    m = re.search(
        r'\s(\d+(?:\.\d+)?)\s*(KBytes|MBytes|GBytes|TBytes|Bytes)\s+'
        r'(\d+(?:\.\d+)?)\s*(Kbits/sec|Mbits/sec|Gbits/sec|bits/sec)\s+.*sender', line)
    if not m:
        raise RuntimeError("iperf sender parse failed: " + line)
    bytes_val = float(m.group(1)) * _UNITS_BYTES[m.group(2)]
    bps_val   = float(m.group(3)) * _UNITS_BITS[m.group(4)]
    return bps_val/1e6, int(bytes_val)

def _iperf_text_summary_old(client_pid, dst_ip, dst_port, duration, extra_args=""):
    out = _ns(client_pid, f"iperf3 -c {shlex.quote(dst_ip)} -p {int(dst_port)} -t {int(duration)} -f m {extra_args}", 
              timeout=duration+60)
    # prefer final [SUM] sender line; fallback to last 'sender'
    lines = out.strip().splitlines()
    target = None
    for ln in lines:
        if "[SUM]" in ln and "sender" in ln:
            target = ln
    if not target:
        for ln in reversed(lines):
            if "sender" in ln:
                target = ln; break
    # after failing to find 'sender', fallback to last line with 'sec' and 'bits/sec'
    if not target:
        for ln in reversed(lines):
            if "sec" in ln and "bits/sec" in ln:
                target = ln; break
    if not target:
        raise RuntimeError("Could not find iperf sender summary line.")
    mbps, bytes_sent = _parse_iperf_sender(target)
    return {"sender_mbps": mbps, "bytes_sent": bytes_sent, "sender_line": target}

# ---- ONOS flow counters helper (expects your get_a_flow_rule_ONOS) ----
def _flow_counters(flow_json):
    try:
        f = flow_json["flows"][0]
        return int(f.get("packets", 0)), int(f.get("bytes", 0))
    except Exception:
        return None, None

# def verify_qos_flow_with_iperf(
#     flow_device_id: str, flow_id: str,
#     queue_device_id: str, queue_port_no: int, queue_id: int,
#     src_ip: str, dst_ip: str, dst_port: int,
#     target_mbps: float,
#     duration_sec: int = 8, parallel: int = 8, tcp_mss: int = 1200,
#     tolerance_pct: float = 10.0,
#     pin_path_flows: list | None = None,   # optional: list of (deviceId, out_port, direction) dicts you push via your push_flow_rule
#     protocol: str = "tcp",                # <-- NEW
#     udp_bw_mbps: float = 50.0,            # <-- NEW (target send rate for UDP)
#     udp_len_bytes: int = 1200,            # <-- NEW (datagram size)
# ):
#     """
#     Returns a dict result and prints a human summary.
#     """
#     # Resolve actors
#     iface = get_iface_for_port(queue_device_id, queue_port_no)
#     client_pid = get_mininet_host_pid(ip_to_host[src_ip])
#     server_pid = get_mininet_host_pid(ip_to_host[dst_ip])

#     proto = protocol.lower()
#     if proto == "tcp":
#         extra_args = f"-P {parallel} -M {tcp_mss}"
#     else:
#         # For UDP you must specify a target rate with -b (pick higher than your cap so the shaper clamps it)
#         extra_args = f"-u -b {udp_bw_mbps}M -l {udp_len_bytes} -P {max(1, parallel)}"

#     # Optional: push pinning flows if provided (expects caller to craft correct selectors)
#     pushed_ids = []
#     if pin_path_flows:
#         for item in pin_path_flows:
#             dev, outp, direction = item["deviceId"], item["out_port"], item["direction"]
#             if direction == "forward":
#                 sel = build_pin_selector_forward(src_ip, dst_ip, dst_port, proto=proto)
#             else:
#                 sel = build_pin_selector_reverse(src_ip, dst_ip, dst_port, proto=proto)
#             flow_body = {
#                 "flows": [{
#                     "priority": 65000,
#                     "isPermanent": True,
#                     "deviceId": dev,
#                     "treatment": {"instructions":[ {"type":"OUTPUT","port": str(outp)} ]},
#                     "selector": sel
#                 }]}
#             fid = push_flow_rule(dev, flow_body)
#             pushed_ids.append((dev, fid))

#     # Best-effort offload guard at egress
#     try: _sudo(f"ethtool -K {shlex.quote(iface)} gro off gso off tso off", timeout=6)
#     except Exception: pass

#     # 1) Snapshots before
#     #print("\nSnapshot 1: before\n")
#     flow_before = get_a_flow_rule_ONOS(flow_device_id, flow_id) or {}
#     f_pkts0, f_bytes0 = _flow_counters(flow_before)
#     q0 = snapshot_queue(iface, queue_id)

#     # 2) Run iperf
#     start_iperf_server(server_pid, dst_port)
#     time.sleep(0.6)
#     t0 = time.time()
#     iptxt = _iperf_text_summary(client_pid, dst_ip, dst_port, duration_sec, extra_args=extra_args)
#     t1 = time.time()
#     stop_iperf_server(server_pid, dst_port)
#     time.sleep(0.8)

#     # 3) Snapshots after
#     #print("\nSnapshot 1: before\n")
#     flow_after = get_a_flow_rule_ONOS(flow_device_id, flow_id) or {}
#     f_pkts1, f_bytes1 = _flow_counters(flow_after)
#     q1 = snapshot_queue(iface, queue_id)

#     # 4) Deltas
#     elapsed = max(0.001, t1 - t0)
#     q_bytes = None if (q0["tx_bytes"] is None or q1["tx_bytes"] is None) else (q1["tx_bytes"] - q0["tx_bytes"])
#     q_mbps  = (q_bytes * 8 / elapsed / 1e6) if q_bytes is not None else None
#     f_pkts_delta = (f_pkts1 - f_pkts0) if (f_pkts0 is not None and f_pkts1 is not None) else None
#     f_bytes_delta = (f_bytes1 - f_bytes0) if (f_bytes0 is not None and f_bytes1 is not None) else None
#     f_mbps = (f_bytes_delta * 8 / elapsed / 1e6) if (f_bytes_delta is not None) else None

#     # 5) Decision
#     rate_ok   = (q_mbps is not None) and (abs(q_mbps - target_mbps) <= (tolerance_pct/100.0)*target_mbps)
#     packets_ok= (f_pkts_delta or 0) > 0
#     verdict   = "PASS" if (rate_ok and packets_ok) else "FAIL"

#     # Print summary
#     print("\n=== QoS FLOW VERIFICATION ===")
#     print(f"Flow {flow_id} @ {flow_device_id}  → selector should match {protocol} dst {dst_port} to {dst_ip}")
#     print(f"Egress iface={iface} queue={queue_id}  target≈{target_mbps:.3f} Mbps  tol=±{tolerance_pct:.0f}%")
#     print(f"iperf sender: {iptxt['sender_mbps']:.3f} Mbps  bytes≈{iptxt['bytes_sent']}")
#     print(f"Queue Δbytes={q_bytes} over {elapsed:.3f}s  → queue_measured≈{(q_mbps or 0):.3f} Mbps")
#     print(f"Flow Δ: packets={f_pkts_delta} bytes={f_bytes_delta}  (flow_measured≈{(f_mbps or 0):.3f} Mbps)")
#     print(f"Queue caps (min/max): {q0['min_rate']} / {q0['max_rate']}  →  {q1['min_rate']} / {q1['max_rate']}")
#     print("VERDICT:", verdict)

#     return {
#         "elapsed_sec": elapsed,
#         "iperf_sender_mbps": iptxt["sender_mbps"],
#         "queue_measured_mbps": q_mbps,
#         "queue_delta_bytes": q_bytes,
#         "flow_delta_packets": f_pkts_delta,
#         "flow_delta_bytes": f_bytes_delta,
#         "flow_measured_mbps": f_mbps,
#         "verdict": verdict
#     }

def ensure_qos_cap(device_id: str, port_no: int, qid: int, min_bps: int, max_bps: int, port_cap_bps: int = 100_000_000):
    iface = get_iface_for_port(device_id, port_no)
    print(f"[INFO] Setting QoS on {iface}: q{qid} min={min_bps} max={max_bps}, root max-rate={port_cap_bps}bps")
    sudo_sh(f"ovs-vsctl --if-exists clear Port {shlex.quote(iface)} qos")
    cmd = (
        "ovs-vsctl "
        f"-- --id=@q create Queue other-config:min-rate={min_bps} other-config:max-rate={max_bps} "
        f"-- --id=@qos create QoS type=linux-htb other-config:max-rate={port_cap_bps} queues:{qid}=@q "
        f"-- set Port {shlex.quote(iface)} qos=@qos"
    )
    print(sudo_sh(cmd))
    print(sudo_sh(f"ovs-appctl -t ovs-vswitchd qos/show {shlex.quote(iface)}"))


# --- NEW [cell 23] ---
# QoS verification helper functions (Ryu-compatible)

# (These OVS/iperf helpers from your original cell 23 are controller-agnostic and correct)
# _UNITS_BYTES, _UNITS_BITS, _sudo, _ns, _parse_iperf_sender, _iperf_text_summary
# (Assume they are copied here)

# --- NEW Helper: Get flow by cookie ---
def get_flow_by_cookie(base, dpid, cookie):
    """
    Fetches all flows for a dpid and returns the one matching the cookie.
    """
    all_flows_dict = get_flows(base, dpid)
    flows_list = all_flows_dict.get(str(dpid), [])
    
    # Use the find_by_cookie helper from [cell 5]
    return find_by_cookie(flows_list, cookie)

# --- NEW Helper: Get flow counters ---
def _ryu_flow_counters(flow_json):
    """
    Extracts packet and byte counters from a Ryu flow rule.
    """
    if flow_json:
        return int(flow_json.get("packet_count", 0)), int(flow_json.get("byte_count", 0))
    return None, None

# --- NEW [cell 24] ---
# (This cell contains helpers from your original topology)
# (HOSTS, SW_OF, HOST_ATTACH are fine)
# (The helpers like choose_dst_for_port, fill_missing_endpoints, etc. are also fine)

# --- NEW [cell 25] ---
# (Contains the main QoS verifier and pinning logic)

def verify_qos_flow_with_iperf(
    flow_device_id: int, flow_cookie: int, # <-- Changed to cookie
    queue_device_id: int, queue_port_no: int, queue_id: int,
    src_ip: str, dst_ip: str, dst_port: int,
    target_mbps: float,
    duration_sec: int = 8, parallel: int = 8, tcp_mss: int = 1200,
    tolerance_pct: float = 10.0,
    pin_path_flows: list | None = None, # This will be built by our new Ryu helper
    protocol: str = "tcp",
    udp_bw_mbps: float = 50.0,
    udp_len_bytes: int = 1200,
):
    """
    Verifies a QoS flow using iperf.
    This version is adapted for Ryu:
    - Uses dpid (int) and flow_cookie (int) to find the flow.
    - Uses Ryu-compatible pinning flows.
    """
    base_url = f"http://{RYU_CONTROLLER_IP}:{RYU_CONTROLLER_PORT}"
    
    # Resolve actors (This logic is OVS/Mininet based, no change needed)
    #iface = get_iface_for_port(f"of:{queue_device_id:016x}", queue_port_no) # get_iface still needs ONOS format
    iface = get_iface_for_port(queue_device_id, queue_port_no)
    client_pid = get_mininet_host_pid(ip_to_host[src_ip])
    server_pid = get_mininet_host_pid(ip_to_host[dst_ip])
    proto = protocol.lower()
    
    if proto == "tcp":
        extra_args = f"-P {parallel} -M {tcp_mss}"
    else:
        extra_args = f"-u -b {udp_bw_mbps}M -l {udp_len_bytes} -P {max(1, parallel)}"

    # --- 1. Install Pinning Flows ---
    # pin_path_flows is the *plan* (a list of dicts)
    # We need to install them and get their cookies for cleanup
    pushed_pin_flows = [] # This will store the actual flow dicts
    if pin_path_flows:
        pushed_pin_flows = install_pins_from_plan_ryu(
            pins=pin_path_flows,
            src_ip=src_ip, dst_ip=dst_ip,
            dst_port=dst_port, protocol=protocol
        )
        print(f"Installed {len(pushed_pin_flows)} pinning flows.")

    try:
        # (Offload guard, iperf server start... all fine)
        try: _sudo(f"ethtool -K {shlex.quote(iface)} gro off gso off tso off", timeout=6)
        except Exception: pass

        # 2) Snapshots before
        flow_before = get_flow_by_cookie(base_url, flow_device_id, flow_cookie)
        f_pkts0, f_bytes0 = _ryu_flow_counters(flow_before)
        q0 = snapshot_queue(iface, queue_id) # OVS command, no change

        # 3) Run iperf
        start_iperf_server(server_pid, dst_port)
        time.sleep(0.6)
        t0 = time.time()
        iptxt = _iperf_text_summary(client_pid, dst_ip, dst_port, duration_sec, extra_args=extra_args)
        t1 = time.time()
        stop_iperf_server(server_pid, dst_port)
        time.sleep(0.8)

        # 4) Snapshots after
        flow_after = get_flow_by_cookie(base_url, flow_device_id, flow_cookie)
        f_pkts1, f_bytes1 = _ryu_flow_counters(flow_after)
        q1 = snapshot_queue(iface, queue_id) # OVS command, no change

        # 5) Deltas (This logic is unchanged)
        elapsed = max(0.001, t1 - t0)
        q_bytes = None if (q0.get("tx_bytes") is None or q1.get("tx_bytes") is None) else (q1["tx_bytes"] - q0["tx_bytes"])
        q_mbps  = (q_bytes * 8 / elapsed / 1e6) if q_bytes is not None else None
        f_pkts_delta = (f_pkts1 - f_pkts0) if (f_pkts0 is not None and f_pkts1 is not None) else None
        f_bytes_delta = (f_bytes1 - f_bytes0) if (f_bytes0 is not None and f_bytes1 is not None) else None
        f_mbps = (f_bytes_delta * 8 / elapsed / 1e6) if (f_bytes_delta is not None) else None

        # 6) Decision (This logic is unchanged)
        rate_ok   = (q_mbps is not None) and (abs(q_mbps - target_mbps) <= (tolerance_pct/100.0)*target_mbps)
        packets_ok= (f_pkts_delta or 0) > 0
        verdict   = "PASS" if (rate_ok and packets_ok) else "FAIL"

        # (Print summary is unchanged)
        print("\n=== QoS FLOW VERIFICATION ===")
        print(f"Flow (cookie:{flow_cookie}) @ DPID {flow_device_id}  → selector should match {protocol} dst {dst_port} to {dst_ip}")
        print(f"Egress iface={iface} queue={queue_id}  target≈{target_mbps:.3f} Mbps  tol=±{tolerance_pct:.0f}%")
        iperf_mbps_str = f"{iptxt['sender_mbps']:.3f}" if iptxt['sender_mbps'] is not None else "N/A"
        iperf_bytes_str = f"{iptxt['bytes_sent']}" if iptxt['bytes_sent'] is not None else "N/A"
        print(f"iperf sender: {iperf_mbps_str} Mbps  bytes≈{iperf_bytes_str}")
        #print(f"iperf sender: {iptxt['sender_mbps']:.3f} Mbps  bytes≈{iptxt['bytes_sent']}")
        print(f"Queue Δbytes={q_bytes} over {elapsed:.3f}s  → queue_measured≈{(q_mbps or 0):.3f} Mbps")
        print(f"Flow Δ: packets={f_pkts_delta} bytes={f_bytes_delta}  (flow_measured≈{(f_mbps or 0):.3f} Mbps)")
        print(f"Queue caps (min/max): {q0['min_rate']} / {q0['max_rate']}  →  {q1['min_rate']} / {q1['max_rate']}")
        print("VERDICT:", verdict)
        
        return {
        "elapsed_sec": elapsed,
        "iperf_sender_mbps": iptxt["sender_mbps"],
        "queue_measured_mbps": q_mbps,
        "queue_delta_bytes": q_bytes,
        "flow_delta_packets": f_pkts_delta,
        "flow_delta_bytes": f_bytes_delta,
        "flow_measured_mbps": f_mbps,
        "verdict": verdict
    }
    
    finally:
        # --- 7. Always Clean Up Pinning Flows ---
        print(f"Cleaning up {len(pushed_pin_flows)} pinning flows...")
        unpin_path_ryu(pushed_pin_flows)

# --- Ryu-compatible Pinning Functions ---

def _pin_selector_ryu(direction: str, protocol: str, src_ip: str, dst_ip: str, dst_port: int):
    """Builds a Ryu-compatible match dictionary."""
    is_udp = protocol.lower() == "udp"
    match = {
        "eth_type": 2048, # 0x0800
        "ip_proto": 17 if is_udp else 6
    }
    
    if direction == "forward":
        match["ipv4_src"] = src_ip
        match["ipv4_dst"] = dst_ip
        if is_udp:
            match["udp_dst"] = int(dst_port)
        else:
            match["tcp_dst"] = int(dst_port)
    else: # reverse
        match["ipv4_src"] = dst_ip
        match["ipv4_dst"] = src_ip
        if is_udp:
            match["udp_src"] = int(dst_port)
        else:
            match["tcp_src"] = int(dst_port)
    return match

def install_pins_from_plan_ryu(pins: list, src_ip: str, dst_ip: str, dst_port: int, protocol: str) -> list:
    """
    Install the pin flows (Ryu format) and return the list of installed flows.
    """
    base_url = f"http://{RYU_CONTROLLER_IP}:{RYU_CONTROLLER_PORT}"
    pushed_flows = []
    
    for p in pins:
        # dpid is integer, e.g., 1
        dev_dpid = int(p["deviceId"].split(":")[-1]) 
        outp = str(p["out_port"])
        direction = p["direction"]
        
        flow_body = {
            "dpid": dev_dpid,
            "priority": 65000, # High priority
            "cookie": secrets.randbits(64), # Unique cookie for temp rules
            "match": _pin_selector_ryu(direction, protocol, src_ip, dst_ip, dst_port),
            "actions": [
                {"type": "OUTPUT", "port": int(outp)}
            ]
        }
        
        try:
            add_flow(base_url, flow_body)
            pushed_flows.append(flow_body) # Store the whole flow for deletion
        except Exception as e:
            print(f"Warning: Failed to install pin flow: {e}")
            
    return pushed_flows

def unpin_path_ryu(pushed_flows: list):
    """
    Remove all pinned rules using the 'delete_flow' helper from [cell 5].
    """
    base_url = f"http://{RYU_CONTROLLER_IP}:{RYU_CONTROLLER_PORT}"
    for flow in pushed_flows:
        delete_flow(base_url, flow)

In [2538]:
# --- New helpers to satisfy the “missing src/dst” rules ---

def choose_dst_for_port(device_id: str, port_no: int) -> str:
    """
    Pick a destination IP that makes forward traffic EXIT on (device_id, port_no).
    Uses your diamond wiring.
    """
    # Final-hop host ports first
    if device_id == SW_OF["1"] and port_no == 3:  # s1 -> h1
        return HOSTS["h1"]
    if device_id == SW_OF["1"] and port_no == 4:  # s1 -> h2
        return HOSTS["h2"]
    if device_id == SW_OF["4"] and port_no == 3:  # s4 -> h3
        return HOSTS["h3"]
    if device_id == SW_OF["4"] and port_no == 4:  # s4 -> h4
        return HOSTS["h4"]

    # Inter-switch egress: choose a host "behind" the far side so packets must traverse this link
    # s1:1->s2 (right), s1:2->s3 (right) → pick a right-side host (h3 default)
    if device_id == SW_OF["1"] and port_no in (1, 2):
        return HOSTS["h3"]
    # s2:2->s4 (right) → pick right-side host; s2:1->s1 (left) → pick left-side host
    if device_id == SW_OF["2"] and port_no == 2:
        return HOSTS["h3"]
    if device_id == SW_OF["2"] and port_no == 1:
        return HOSTS["h1"]
    # s3:2->s4 (right) → right host; s3:1->s1 (left) → left host
    if device_id == SW_OF["3"] and port_no == 2:
        return HOSTS["h3"]
    if device_id == SW_OF["3"] and port_no == 1:
        return HOSTS["h1"]
    # s4 inter-switch (rare in your tests): port1->s2 (left) pick left host; port2->s3 (left) pick left host
    if device_id == SW_OF["4"] and port_no in (1, 2):
        return HOSTS["h1"]

    # Fallback
    return HOSTS["h3"]

def fill_missing_endpoints(plan: dict, device_id: str, port_no: int) -> tuple[str, str]:
    """
    Apply your policy:
      - If only dst is present → choose src that forces egress at (device,port)
      - If only src is present → choose dst that forces egress at (device,port)
      - If both missing → pick both to force egress at (device,port)
      - Ensure src != dst
    """
    src = plan.get("src_ip")
    dst = plan.get("dst_ip")

    # Normalize empty strings to None
    src = src or None
    dst = dst or None

    if dst is None:
        dst = choose_dst_for_port(device_id, port_no)
    if src is None:
        src = choose_src_for_port(device_id, port_no)

    # If they accidentally collide, flip src to the opposite side
    if src == dst:
        # If dst is on right side, move src to left; else move to right
        try:
            dst_edge, _ = _edge_for_ip(dst)
        except Exception:
            dst_edge = SW_OF["4"]  # assume right if unknown
        src = HOSTS["h1"] if dst_edge in (SW_OF["3"], SW_OF["4"]) else HOSTS["h3"]

    return src, dst

def _strip32(ip: str) -> str:
    return ip.split("/", 1)[0]

def find_edge_for_ip(dst_ip: str):
    """Return (deviceId, portNo) for the host that owns dst_ip.
       Prefer static map; fall back to ONOS /hosts if needed."""
    ip = _strip32(dst_ip)
    if ip in HOST_ATTACH:
        return HOST_ATTACH[ip]
    # Optional fallback via ONOS:
    try:
        js = onos_get(f"hosts?ip={ip}")
        lst = js.get("hosts", js if isinstance(js, list) else [])
        if lst:
            locs = lst[0].get("locations") or lst[0].get("location") or []
            if locs:
                dev = locs[0].get("elementId") or locs[0].get("device") or locs[0].get("element")
                port = int(locs[0].get("port"))
                return dev, port
    except Exception:
        pass
    raise RuntimeError(f"Could not infer edge device/port for dst_ip={dst_ip}")

def infer_enforcement_point(plan):
    """
    Decide where to enforce if device/port not fully specified.
    Preference:
      1) If plan.device_id & plan.port_no given → use those
      2) Else enforce at the final hop to the destination host (works for h1..h4).
    """
    device_id, port_no = plan.get("device_id"), plan.get("port_no")
    if device_id and port_no:
        return device_id, port_no
    return find_edge_for_ip(plan["dst_ip"])

def _edge_for_ip(ip: str):
    ip = ip.split("/", 1)[0]
    if ip in HOST_ATTACH:
        return HOST_ATTACH[ip]
    # Optional: try ONOS if not in static map (kept simple)
    try:
        js = onos_get(f"hosts?ip={ip}")
        lst = js.get("hosts", js if isinstance(js, list) else [])
        if lst:
            loc = (lst[0].get("locations") or lst[0].get("location") or [])[0]
            return loc["elementId"], int(loc["port"])
    except Exception:
        pass
    raise RuntimeError(f"No edge mapping for IP {ip}")

def _is(dev: str, n: str) -> bool:
    """dev == SW_OF[n]?"""
    return dev == SW_OF[n]

def choose_src_for_port(device_id: str, port_no: int) -> str:
    """
    Pick a source IP that will make *forward* traffic egress on (device_id, port_no).
    This aligns the traffic direction with your enforcement point so your queue is actually exercised.
    """
    # s1: final-host ports 3/4 → pick a source on the right (h3)
    if _is(device_id, "1"):
        if port_no in (3, 4):        # s1 -> h1/h2
            return HOSTS["h3"]
        elif port_no in (1, 2):      # s1 -> s2/s3 (inter-switch)
            return HOSTS["h1"]       # left host causes egress at s1
    # s2: port2 to s4 → src on left; port1 to s1 → src on right
    if _is(device_id, "2"):
        return HOSTS["h1"] if port_no == 2 else HOSTS["h3"]
    # s3: port2 to s4 → src on left; port1 to s1 → src on right
    if _is(device_id, "3"):
        return HOSTS["h1"] if port_no == 2 else HOSTS["h3"]
    # s4: final-host ports 3/4 → src on left; inter-switch (1/2) → src on right
    if _is(device_id, "4"):
        if port_no in (3, 4):        # s4 -> h3/h4
            return HOSTS["h1"]
        elif port_no in (1, 2):      # s4 -> s2/s3
            return HOSTS["h3"]
    # Fallback
    return HOSTS["h1"]

def make_pin_path_flows(device_id: str, port_no: int,
                        src_ip: str, dst_ip: str, dst_port: int,
                        protocol: str = "tcp"):
    """
    Build a pinning plan so src_ip -> dst_ip forward traffic *must* traverse (device_id, port_no).

    Returns a list of dicts: [{"deviceId":..., "out_port":..., "direction":"forward|reverse"}, ...]
    - For TCP: includes reverse pins (ACK path) to reduce variability.
    - For UDP: forward-only pins (reverse is optional, skipped here).
    """
    proto = protocol.lower()
    is_udp = (proto == "udp")

    pins = []
    src_edge_dev, src_edge_port = _edge_for_ip(src_ip)
    dst_edge_dev, dst_edge_port = _edge_for_ip(dst_ip)

    def add(dev, outp, direction):
        pins.append({"deviceId": dev, "out_port": int(outp), "direction": direction})

    # ----- cases by enforcement point -----

    # s4 final hop to host (port 3 or 4: h3/h4)
    if _is(device_id, "4") and port_no in (3, 4):
        # Forward: steer from the left toward s4 via s1->s3->s4
        # If source is already on the right (h3/h4), nothing to pin on the way in.
        if _is(src_edge_dev, "1"):
            #add(SW_OF["1"], 2, "forward")   # s1 -> s3
            #add(SW_OF["3"], 2, "forward")   # s3 -> s4
            add(SW_OF["1"], 1, "forward")   # s1 -> s2 (port 1)
            add(SW_OF["2"], 2, "forward")   # s2 -> s4 (port 2)
        elif _is(src_edge_dev, "4"):
            pass  # already on s4 side
        elif _is(src_edge_dev, "2"):
            add(SW_OF["2"], 2, "forward")   # s2 -> s4
        elif _is(src_edge_dev, "3"):
            add(SW_OF["3"], 2, "forward")   # s3 -> s4

        # Reverse (TCP): s4 -> s3 -> s1 -> host(src)
        if not is_udp:
            if _is(src_edge_dev, "1"):
                #add(SW_OF["4"], 2, "reverse")   # s4 -> s3
                #add(SW_OF["3"], 1, "reverse")   # s3 -> s1
                #add(SW_OF["1"], src_edge_port, "reverse")  # s1 -> h1/h2
                add(SW_OF["4"], 1, "reverse")   # s4 -> s2 (port 1)
                add(SW_OF["2"], 1, "reverse")   # s2 -> s1 (port 1)
                add(SW_OF["1"], src_edge_port, "reverse")  # s1 -> h1/h2
            elif _is(src_edge_dev, "4"):
                add(SW_OF["4"], src_edge_port, "reverse")  # local s4 -> h3/h4

    # s1 final hop to host (port 3 or 4: h1/h2)
    elif _is(device_id, "1") and port_no in (3, 4):
        # Forward: steer from the right toward s1 via s4->s3->s1
        if _is(src_edge_dev, "4"):
            add(SW_OF["4"], 2, "forward")   # s4 -> s3
            add(SW_OF["3"], 1, "forward")   # s3 -> s1
        elif _is(src_edge_dev, "1"):
            pass  # already on s1 side

        # Reverse (TCP): s1 -> s3 -> s4 -> host(src)
        if not is_udp:
            if _is(src_edge_dev, "4"):
                add(SW_OF["1"], 2, "reverse")   # s1 -> s3
                add(SW_OF["3"], 2, "reverse")   # s3 -> s4
                add(SW_OF["4"], src_edge_port, "reverse")  # s4 -> h3/h4
            elif _is(src_edge_dev, "1"):
                add(SW_OF["1"], src_edge_port, "reverse")  # local s1 -> h1/h2

    # s1 inter-switch egress
    elif _is(device_id, "1") and port_no == 2:
        # Forward: use s1->s3->s4 path
        add(SW_OF["1"], 2, "forward")
        add(SW_OF["3"], 2, "forward")
        # Reverse (TCP): s4 -> s3 -> s1 -> host(src)
        if not is_udp:
            add(SW_OF["4"], 2, "reverse")
            add(SW_OF["3"], 1, "reverse")
            add(SW_OF["1"], src_edge_port, "reverse")
    elif _is(device_id, "1") and port_no == 1:
        # Forward: use s1->s2->s4 path
        add(SW_OF["1"], 1, "forward")
        add(SW_OF["2"], 2, "forward")
        # Reverse (TCP): s4 -> s2 -> s1 -> host(src)
        if not is_udp:
            add(SW_OF["4"], 1, "reverse")
            add(SW_OF["2"], 1, "reverse")
            add(SW_OF["1"], src_edge_port, "reverse")

    # s2 inter-switch egress
    elif _is(device_id, "2") and port_no == 2:
        # Forward: s1 -> s2 -> s4
        add(SW_OF["1"], 1, "forward")
        add(SW_OF["2"], 2, "forward")
        if not is_udp:
            add(SW_OF["4"], 1, "reverse")
            add(SW_OF["2"], 1, "reverse")
            add(SW_OF["1"], src_edge_port, "reverse")
    elif _is(device_id, "2") and port_no == 1:
        # Forward: s4 -> s2 -> s1
        add(SW_OF["4"], 1, "forward")
        add(SW_OF["2"], 1, "forward")
        if not is_udp:
            add(SW_OF["1"], 1, "reverse")
            add(SW_OF["2"], 2, "reverse")
            add(SW_OF["4"], src_edge_port, "reverse")

    # s3 inter-switch egress
    elif _is(device_id, "3") and port_no == 2:
        # Forward: s1 -> s3 -> s4
        add(SW_OF["1"], 2, "forward")
        add(SW_OF["3"], 2, "forward")
        if not is_udp:
            add(SW_OF["4"], 2, "reverse")
            add(SW_OF["3"], 1, "reverse")
            add(SW_OF["1"], src_edge_port, "reverse")
    elif _is(device_id, "3") and port_no == 1:
        # Forward: s4 -> s3 -> s1
        add(SW_OF["4"], 2, "forward")
        add(SW_OF["3"], 1, "forward")
        if not is_udp:
            add(SW_OF["1"], 2, "reverse")
            add(SW_OF["3"], 2, "reverse")
            add(SW_OF["4"], src_edge_port, "reverse")

    # s4 inter-switch egress (rare in your tests but supported)
    elif _is(device_id, "4") and port_no in (1, 2):
        # To use s4->(2?3?) link in *forward* direction, src should be on the right.
        # Forward: none if src is already on s4; else steer toward s4 via right path
        if _is(src_edge_dev, "1"):
            add(SW_OF["1"], 2, "forward")  # prefer via s3
            add(SW_OF["3"], 2, "forward")
        # Reverse (TCP): symmetric back to src
        if not is_udp:
            if port_no == 1:
                add(SW_OF["4"], 1, "reverse"); add(SW_OF["2"], 1, "reverse")
                add(SW_OF["1"], src_edge_port, "reverse")
            else:
                add(SW_OF["4"], 2, "reverse"); add(SW_OF["3"], 1, "reverse")
                add(SW_OF["1"], src_edge_port, "reverse")

    else:
        # Fallback: route via s1->s3->s4 with TCP reverse symmetry
        add(SW_OF["1"], 2, "forward")
        add(SW_OF["3"], 2, "forward")
        if not is_udp:
            add(SW_OF["4"], 2, "reverse")
            add(SW_OF["3"], 1, "reverse")
            add(SW_OF["1"], src_edge_port, "reverse")

    # For UDP, we intentionally omit reverse pins (ACK-less).
    # if is_udp:
    #     pins = [p for p in pins if p["direction"] == "forward"]

    # --- Never overshadow the QoS rule on the enforcement port ---
    # Drop any FORWARD pin that sits on the enforcement device+port.
    pins = [p for p in pins
            if not (p["direction"] == "forward"
                    and p["deviceId"] == device_id
                    and int(p["out_port"]) == int(port_no))]

    # For UDP, keep forward-only (already done above)
    if is_udp:
        pins = [p for p in pins if p["direction"] == "forward"]

    # De-duplicate [(dev,port,direction)] triples, keep first
    seen, uniq = set(), []
    for p in pins:
        key = (p["deviceId"], int(p["out_port"]), p["direction"])
        if key in seen: 
            continue
        seen.add(key); uniq.append(p)
    return uniq
    #return pins

def normalize_device_id(dev: str | None) -> str | None:
    if not dev:
        return dev
    dev = dev.strip()

    # openflow:<decimal>
    m = re.fullmatch(r'openflow:(\d+)', dev, flags=re.I)
    if m:
        return f"of:{int(m.group(1)):016x}"

    # openflow:<hex> or openflow:0x<hex>
    m = re.fullmatch(r'openflow:(?:0x)?([0-9a-fA-F]+)', dev, flags=re.I)
    if m:
        return f"of:{int(m.group(1), 16):016x}"

    # of:<decimal>
    m = re.fullmatch(r'of:(\d+)', dev, flags=re.I)
    if m:
        return f"of:{int(m.group(1)):016x}"

    # of:<hex already>
    if dev.lower().startswith("of:"):
        return "of:" + dev[3:].lower()

    return dev

def parse_intent_text(slicing_info, src_ip, dst_ip):

                slicing_queue_id = slicing_info['queue_id'] if (slicing_info['queue_id']) != "" else 1
                proto = slicing_info['traffic_type'] if "udp" in slicing_info['traffic_type'] else ("tcp" if "tcp" in slicing_info['traffic_type'] or "http" in slicing_info['traffic_type'] else "tcp")
                slicing_l4_port = slicing_info['l4_port'] if slicing_info['l4_port'] != "" else 80
                port_no = extract_port_number(slicing_info['port_id'])
                device_id = extract_switch_id(slicing_info['switch_id'])
                device_id = normalize_device_id (device_id)

                return {
                    "protocol": proto,
                    "dst_port": slicing_l4_port,
                    "device_id": device_id,   # may be None
                    "port_no": port_no,       # may be None, it means output interface number
                    "queue_id": slicing_queue_id,
                    "src_ip": src_ip,         # may be None
                    "dst_ip": dst_ip,         # may be None
                }

# [cell 2023]
# PASTE THIS ENTIRE FUNCTION, REPLACING THE OLD ONE

def intent_to_verifier_args(slicing_info, old_src_ip, old_dst_ip, main_flow_cookie: int, target_mbps: float = 4.0, translated_flow_rule: dict = None):
    """
    Parses the slicing info and the main flow rule to create the
    correct arguments for the verify_qos_flow_with_iperf function.
    
    This version includes the logic to detect 'in_port' from the 
    translated_flow_rule and lock the test's src_ip to the correct host.
    """
    
    # 1. Parse the text intent
    plan = parse_intent_text(slicing_info, old_src_ip, old_dst_ip)
    
    # 2. Find the enforcement point (device/port for the queue)
    # (infer_enforcement_point is fine)
    device_id, port_no = infer_enforcement_point(plan)
    plan["device_id"], plan["port_no"] = device_id, port_no

    # --- START CORRECTED LOGIC ---
    # Check if the main rule has an 'in_port' and use it to find the *correct* source host
    if translated_flow_rule:
        match = translated_flow_rule.get("match", {})
        in_port = match.get("in_port")
        
        if in_port:
            # We have an in_port. Find which host connects to this (device, in_port).
            # HOST_ATTACH is in {ip: (dev_str, port_num)} format
            # We need to build a reverse map: {(dev_str, port_num): ip}
            
            # Build the reverse map (HOST_ATTACH is defined in this cell)
            port_to_ip = {v: k for k, v in HOST_ATTACH.items()}
            
            # Check if this (device, in_port) corresponds to a known host
            # The 'device_id' variable IS the correct 'of:...' string (e.g., "of:0000000000000001")
            host_ip = port_to_ip.get( (device_id, int(in_port)) )
            
            if host_ip:
                print(f"[INFO] Rule has in_port:{in_port}, locking source host to {host_ip}")
                plan["src_ip"] = host_ip # Set the source IP
            else:
                # This can happen if in_port is an inter-switch port
                print(f"[WARN] Rule has in_port:{in_port}, but no host is mapped to ({device_id}, {in_port}).")
    # --- END CORRECTED LOGIC ---

    # 3. Fill any remaining missing endpoints
    # This will now use plan["src_ip"] if we set it, or guess if we didn't.
    src_ip, dst_ip = fill_missing_endpoints(plan, device_id, port_no)
    plan["src_ip"], plan["dst_ip"] = src_ip, dst_ip

    # 4. Convert 'of:...' string to integer dpid for Ryu functions
    # (split on ':', take last part, convert from hex)
    dpid = int(device_id.split(":")[-1], 16) 

    # 5. Build verifier args
    args = {
        "flow_device_id":  dpid,
        "flow_cookie":     main_flow_cookie,
        "queue_device_id": dpid,
        "queue_port_no":   port_no,
        "queue_id":        plan["queue_id"],
        "src_ip":          src_ip,
        "dst_ip":          dst_ip,
        "dst_port":        plan["dst_port"],
        "target_mbps":     target_mbps,
        "protocol":        plan["protocol"]
    }

    # 6. Build Pin plan
    pins_plan = make_pin_path_flows(
        device_id, 
        port_no, 
        src_ip, 
        dst_ip, 
        plan["dst_port"],  # <-- Fixed potential typo here too
        plan["protocol"]
    )
    
    return args, pins_plan, plan

def intent_to_verifier_args_old2(slicing_info, old_src_ip, old_dst_ip, main_flow_cookie: int, target_mbps: float = 4.0, translated_flow_rule: dict = None):
    
    # (parse_intent_text is fine)
    plan = parse_intent_text(slicing_info, old_src_ip, old_dst_ip)
    
    # (infer_enforcement_point is fine)
    device_id, port_no = infer_enforcement_point(plan)
    plan["device_id"], plan["port_no"] = device_id, port_no

    # --- START NEW LOGIC ---
    # Check if the main rule has an 'in_port' and use it to find the *correct* source host
    if translated_flow_rule:
        match = translated_flow_rule.get("match", {})
        in_port = match.get("in_port")
        
        if in_port:
            # We have an in_port. Find which host connects to this (device, in_port).
            # HOST_ATTACH is in {ip: (dev_str, port_num)} format
            # We need to build a reverse map: {(dev_str, port_num): ip}
            
            # (SW_OF and HOST_ATTACH are defined earlier in this cell)
            rule_device_str = SW_OF.get(str(device_id.split(":")[-1])) # "of:..."
            
            # Build the reverse map
            port_to_ip = {v: k for k, v in HOST_ATTACH.items()}
            
            # Check if this (device, in_port) corresponds to a known host
            host_ip = port_to_ip.get( (rule_device_str, int(in_port)) )
            
            if host_ip:
                print(f"[INFO] Rule has in_port:{in_port}, locking source host to {host_ip}")
                plan["src_ip"] = host_ip # Set the source IP
    
    # --- END NEW LOGIC ---

    # (fill_missing_endpoints is fine)
    # This will now use plan["src_ip"] if we set it, or guess if we didn't.
    src_ip, dst_ip = fill_missing_endpoints(plan, device_id, port_no)
    plan["src_ip"], plan["dst_ip"] = src_ip, dst_ip

    # Convert 'of:...' string to integer dpid
    dpid = int(device_id.split(":")[-1])

    # 3) Build verifier args
    # (Rest of the function is unchanged)
    args = {
        "flow_device_id":  dpid,
        "flow_cookie":     main_flow_cookie, # <-- PASS COOKIE
        "queue_device_id": dpid,
        "queue_port_no":   port_no,
        "queue_id":        plan["queue_id"],
        "src_ip":          src_ip,
        "dst_ip":          dst_ip,
        "dst_port":        plan["dst_port"],
        "target_mbps":     target_mbps,
        "protocol":        plan["protocol"]
    }

    # 4) Build Pin plan (This logic is OVS-agnostic and fine)
    pins_plan = make_pin_path_flows(device_id, port_no, src_ip, dst_ip, plan["dst_port"], plan["protocol"])                                                                            
    return args, pins_plan, plan
    

def intent_to_verifier_args_old(slicing_info, old_src_ip, old_dst_ip, main_flow_cookie: int, target_mbps: float = 4.0):
    
    # (parse_intent_text is fine)
    plan = parse_intent_text(slicing_info, old_src_ip, old_dst_ip)

    # (infer_enforcement_point is fine)
    device_id, port_no = infer_enforcement_point(plan)
    plan["device_id"], plan["port_no"] = device_id, port_no
    
    # (fill_missing_endpoints is fine)
    src_ip, dst_ip = fill_missing_endpoints(plan, device_id, port_no)
    plan["src_ip"], plan["dst_ip"] = src_ip, dst_ip

    # Convert 'of:...' string to integer dpid
    dpid = int(device_id.split(":")[-1])

    # 3) Build verifier args
    args = {
        "flow_device_id":  dpid,
        "flow_cookie":     main_flow_cookie, # <-- PASS COOKIE
        "queue_device_id": dpid,
        "queue_port_no":   port_no,
        "queue_id":        plan["queue_id"],
        "src_ip":          src_ip,
        "dst_ip":          dst_ip,
        "dst_port":        plan["dst_port"],
        "target_mbps":     target_mbps,
        "protocol":        plan["protocol"]
    }

    # 4) Build Pin plan (This logic is OVS-agnostic and fine)
    pins_plan = make_pin_path_flows(device_id, port_no, src_ip, dst_ip, plan["dst_port"], plan["protocol"])
    return args, pins_plan, plan

def _pin_selector(direction: str, protocol: str, src_ip: str, dst_ip: str, dst_port: int):
    """Build protocol-aware 5-tuple for forward/reverse."""
    is_udp = protocol.lower() == "udp"
    crit = [
        {"type":"ETH_TYPE","ethType":"0x800"},
        {"type":"IP_PROTO","protocol": 17 if is_udp else 6},
    ]
    if direction == "forward":
        crit += [
            {"type":"IPV4_SRC","ip": f"{src_ip}/32"},
            {"type":"IPV4_DST","ip": f"{dst_ip}/32"},
            ({"type":"UDP_DST","udpPort": dst_port} if is_udp else {"type":"TCP_DST","tcpPort": dst_port}),
        ]
    else:  # reverse
        crit += [
            {"type":"IPV4_SRC","ip": f"{dst_ip}/32"},
            {"type":"IPV4_DST","ip": f"{src_ip}/32"},
            ({"type":"UDP_SRC","udpPort": dst_port} if is_udp else {"type":"TCP_SRC","tcpPort": dst_port}),
        ]
    return {"criteria": crit}

In [2539]:
# intent = "Forward TCP traffic on port 80 destined for 10.0.1.3 via interface 3, assigning it to queue 1 for prioritized handling in switch 4."
# intent = "In switch 4, traffic destined for 10.0.1.4 should use port 4."
# intent =  "In switch 4, block all IPv4 traffic from 10.0.1.1 to 10.0.1.4 with a high priority, ensuring the switch operates as a firewall."
# intent = "Drop all traffic from 10.0.1.3 on switch 2 while forwarding all other traffic normally."
# intent = "If interface 1 on node 2 receives UDP to port 80, pass via port 2, queue 1."
# intent = "If switch 1 receives TCP on port 4 to port 80, pass via interface 1, queue 0."
# intent = "In node 1, traffic destined for 10.0.1.2 should use port 4."
# intent = "In switch 2, traffic from port 1 should pass through port 2."
# intent = "Traffic from port 2 of switch 2 to 10.0.1.1 should use interface 1."
intent = "In switch 4, traffic from 10.0.1.1 to 10.0.1.4 should use output interface 4."

In [ ]:
current_time = time.time()

slicing_info = run_LLM_Slice(intent)

#print(slicing_info)

#create_two_queue_for_switch_handler(slicing_info)

deployment_status, flow_id, device_id, translated_flow_rule, operational_flow_rule = end_to_end_IBN(intent)

if 'use_queue' in slicing_info and slicing_info['use_queue'] == 1:
    #Creating one queue
    expected_queue_rate_mbps = 4.0
    port_max = 100_000_000  # 100 Mbps cap for the port QoS root
    min_rate = max_rate = int(expected_queue_rate_mbps * 1_000_000)
    slicing_queue_id = slicing_info['queue_id'] if (slicing_info['queue_id']) != "" else 1
    proto = slicing_info['traffic_type'] if "udp" in slicing_info['traffic_type'] else ("tcp" if "tcp" in slicing_info['traffic_type'] or "http" in slicing_info['traffic_type'] else "tcp")
    slicing_l4_port = slicing_info['l4_port'] if slicing_info['l4_port'] != "" else 80
    port_no = extract_port_number(slicing_info['port_id'])
    
    ensure_qos_cap(device_id, port_no, slicing_queue_id, min_bps=min_rate, max_bps=max_rate, port_cap_bps=100_000_000)

global llm_caller_flag

if (deployment_status == True):
        proc_time_s = (time.time() - current_time)
        print("\n\nSuccessfully translated and installed the rule in Ryu SDN Controller. Time taken: ", proc_time_s)
        src_host, dst_host, src_ip, dst_ip = extract_host_and_ip_ryu(translated_flow_rule)
        flow_rule_type, flow_rule_specificity = classify_ryu_flow_rule(translated_flow_rule)
        
        # Example of appending an intent to IntentStore
        append_intent_to_store(
            "IntentStore_Ryu.jsonl",
            nl_intent=intent,
            json_flow_rule=translated_flow_rule,
            device_id=device_id,
            flow_id=flow_id,
            intent_type=flow_rule_type,
            intent_specificity=flow_rule_specificity
            )
        
        if (flow_rule_type== "security"):
            #ping_count, candidate_src_ip, candidate_dst_ip, ping_output = ryu_assurance_for_security_intent(src_ip, dst_ip)
            ping_count, candidate_src_ip, candidate_dst_ip, ping_output = ryu_assurance_for_security_intent(src_ip, dst_ip, device_id)
            if (llm_caller_flag == 1):
                print("\nAsking LLM to generate corrective actions...")
                assurance_LLM_prompt = generate_corrective_action_prompt_ryu(intent, operational_flow_rule, device_id, ping_count,
                                    candidate_src_ip, candidate_dst_ip, ping_output)
                llm_response = Run_assurance_LLM (assurance_LLM_prompt)
                print(llm_response)
                parse_and_execute_corrective_actions_ryu(operational_flow_rule, llm_response, device_id)


        elif (flow_rule_type== "qos"):
            
            target_mbps = 4.0 # Set your target
            
            # flow_id from end_to_end_IBN is the cookie
            main_flow_cookie = flow_id 
            
            # 1) Resolve intent
            # Pass the cookie to the args builder
            #args, pins_plan, plan = intent_to_verifier_args(slicing_info, src_ip, dst_ip, main_flow_cookie, target_mbps)
            args, pins_plan, plan = intent_to_verifier_args(
                slicing_info, 
                src_ip, 
                dst_ip, 
                main_flow_cookie, 
                target_mbps,
                translated_flow_rule  # <-- ADD THIS ARGUMENT
            )
            protocol = plan["protocol"]
            dst_port = args["dst_port"]

            # (No need to find flow_id, we already have it)

            # 3) Install pin flows (this is now done *inside* the verifier)

            #################################################################
            # --- START FINAL DEBUGGING BLOCK (v2 - Robust Filter) ---
            #################################################################
            # print("\n[DEBUG] --- STARTING FINAL DEBUGGING (v2) ---")
            
            # # --- Get parameters ---
            # debug_src_ip = args["src_ip"]
            # debug_dst_ip = args["dst_ip"]
            # debug_dst_port = args["dst_port"]
            # debug_client_pid = get_mininet_host_pid(ip_to_host[debug_src_ip])
            # debug_server_pid = get_mininet_host_pid(ip_to_host[debug_dst_ip])

            # print(f"[DEBUG] Client: h1 (PID {debug_client_pid}) -> Server: h3 (PID {debug_server_pid})")
            # print(f"[DEBUG] Pinning plan has {len(pins_plan)} flows.")

            # pushed_pins = []
            # tcpdump_running = False
            # try:
            #     # --- STEP 1: Install Pins ---
            #     pushed_pins = install_pins_from_plan_ryu(
            #         pins_plan, debug_src_ip, debug_dst_ip, debug_dst_port, protocol
            #     )
            #     print(f"[DEBUG] STEP 1: Successfully installed {len(pushed_pins)} pinning flows.")
                
            #     # --- STEP 2: L3 Ping Test ---
            #     print("[DEBUG] STEP 2: Testing L3 connectivity (ping h1 -> h3)...")
            #     ping_out, ping_err = run_in_host(debug_client_pid, f"ping -c 3 {debug_dst_ip}", timeout=5)
            #     if "0% packet loss" in ping_out:
            #         print("[DEBUG] PING (h1->h3) SUCCESS: Default path is working.")
            #     else:
            #         print(f"[DEBUG] PING (h1->h3) FAILED. Basic connectivity is down.")

            #     # --- STEP 3: Start Server ---
            #     print("[DEBUG] STEP 3: Starting iperf3 server on h3...")
            #     start_iperf_server(debug_server_pid, debug_dst_port)
            #     print("[DEBUG] Server started.")
                
            #     # --- STEP 4: Find and Start tcpdump ---
            #     print("[DEBUG] STEP 4: Finding and starting tcpdump on h3...")
                
            #     # 4a. Find the path for tcpdump using 'which'
            #     which_out, which_err = run_in_host(debug_server_pid, "/usr/bin/which tcpdump", timeout=5)
            #     tcpdump_path = which_out.strip()
                
            #     if not tcpdump_path or "no tcpdump" in tcpdump_path or "not found" in tcpdump_path:
            #         print("[DEBUG] FATAL: 'tcpdump' not found on h3. Cannot complete L4 debugging.")
            #     else:
            #         print(f"[DEBUG] 'tcpdump' found at: {tcpdump_path}")
                    
            #         # 4b. Start tcpdump with the ROBUST bit-masking filter
            #         # This filter checks for 'tcp port 80' AND 'the SYN flag is set'
                    
            #         # --- *** THIS IS THE ONLY LINE THAT CHANGED *** ---
            #         tcpdump_cmd = f"nohup {tcpdump_path} -i h3-eth0 -c 3 '(tcp port 80) and (tcp[tcpflags] & tcp-syn != 0)' > /tmp/h3_tcpdump.log 2>&1 &"
                    
            #         run_in_host(debug_server_pid, tcpdump_cmd, timeout=5)
            #         tcpdump_running = True
            #         time.sleep(1) # Give tcpdump time to start

            #         # --- STEP 5: Run Netcat ---
            #         print("[DEBUG] STEP 5: Running netcat (h1 -> h3:80)...")
            #         nc_cmd = f"/bin/nc -zvw 4 {debug_dst_ip} {debug_dst_port}"
            #         nc_out, nc_err = run_in_host(debug_client_pid, nc_cmd, timeout=7)

            #         if "succeeded" in nc_err or "Connected" in nc_err:
            #             print(f"[DEBUG] NETCAT SUCCESS: Port {debug_dst_port} is open.")
            #         else:
            #             print(f"[DEBUG] NETCAT FAILED: Port {debug_dst_port} is closed or unreachable.")
            #             print(f"[DEBUG] NETCAT STDOUT: {nc_out}")
            #             print(f"[DEBUG] NETCAT STDERR: {nc_err}")

            #         # --- STEP 6: Check tcpdump results ---
            #         print("[DEBUG] STEP 6: Checking tcpdump results from h3...")
            #         time.sleep(1) # wait for logs
            #         if tcpdump_running:
            #             run_in_host(debug_server_pid, "killall tcpdump", timeout=5) # Stop the capture
                    
            #         dump_log, _ = run_in_host(debug_server_pid, "cat /tmp/h3_tcpdump.log", timeout=5)
            #         print(f"[DEBUG] H3 TCPDUMP LOG:\n{dump_log}")
                    
            #         # Check for the "listening on..." and "packets captured" lines
            #         if ("packets captured" in dump_log and "0 packets" not in dump_log):
            #              print("[DEBUG] ANALYSIS: SUCCESS! The SYN packet ARRIVED at h3.")
            #              print("[DEBUG] ROOT CAUSE: The problem is the REVERSE path (SYN-ACK from h3 to h1).")
            #         elif ("listening on" in dump_log):
            #              print("[DEBUG] ANALYSIS: FAILED! The SYN packet did NOT ARRIVE at h3.")
            #              print("[DEBUG] (tcpdump ran, but 0 packets were captured)")
            #              print("[DEBUG] ROOT CAUSE: The FORWARD path (h1->s1->s3->s4) is broken.")
            #         else:
            #              print("[DEBUG] ANALYSIS: FAILED! The tcpdump command failed to run correctly.")
            #              print("[DEBUG] (The log does not contain expected 'listening' or 'captured' output)")

            # except Exception as e:
            #     print(f"[DEBUG] An exception occurred during debugging: {e}")
            
            # finally:
            #     # --- Always cleanup ---
            #     print("[DEBUG] STEP 7: Cleaning up...")
            #     unpin_path_ryu(pushed_pins)
            #     stop_iperf_server(debug_server_pid, debug_dst_port)
            #     if tcpdump_running:
            #         run_in_host(debug_server_pid, "killall tcpdump || true", timeout=5)
            #     print("[DEBUG] --- MANUAL DEBUGGING COMPLETE ---")
            #################################################################
            # --- END FINAL DEBUGGING BLOCK (v2) ---
            #################################################################


            try:
                # 4) Call the verifier
                if protocol == "udp":
                    verify_qos_flow_with_iperf(
                        flow_device_id=args["flow_device_id"],
                        flow_cookie=args["flow_cookie"], 
                        queue_device_id=args["queue_device_id"],
                        queue_port_no=args["queue_port_no"],
                        queue_id=args["queue_id"],
                        src_ip=args["src_ip"],
                        dst_ip=args["dst_ip"],
                        dst_port=dst_port,
                        target_mbps=args["target_mbps"],
                        protocol="udp",
                        pin_path_flows=pins_plan,
                        
                        # --- Explicit iperf parameters for UDP ---
                        duration_sec=8,
                        parallel=1, # <-- Explicitly set to 1 for UDP
                        tolerance_pct=10.0,
                        udp_bw_mbps=50.0,
                        udp_len_bytes=1200
                    )
                else:
                    # TCP
                    verify_qos_flow_with_iperf(
                        flow_device_id=args["flow_device_id"],
                        flow_cookie=args["flow_cookie"], 
                        queue_device_id=args["queue_device_id"],
                        queue_port_no=args["queue_port_no"],
                        queue_id=args["queue_id"],
                        src_ip=args["src_ip"],
                        dst_ip=args["dst_ip"],
                        dst_port=dst_port,
                        target_mbps=args["target_mbps"],
                        protocol="tcp",
                        pin_path_flows=pins_plan,
                        
                        # --- Explicit iperf parameters for TCP ---
                        duration_sec=8,
                        parallel=8,
                        tcp_mss=1200,
                        tolerance_pct=10.0
                    )
            finally:
                # The verifier function has its own 'finally'
                # block to clean up pin flows.
                pass    
             
        elif (flow_rule_type== "forwarding"):
             ONOS_assurance_for_forwarding_intent(src_host, dst_host, src_ip, dst_ip)

        elapsed_time = (time.time() - current_time)
        print("\nTime taken for end-to-end IBN: ",  round(elapsed_time,2))

elif (flow_id == "Tie"):
    print("\n\nReport to the operator about this conflict resolution issue. Need adjustment to conflict resolution policy.\n")

elif (flow_id == "existing_rule_win"):
    print("\n\nThe new intent conflicts with an existing one having a higher priority according to current policy, hence the new intent was not installed. See the existing flow rule that conflicts.\n")

else:
    print("\n\nLLM failed to produce meaningful response. Either update context example or model.\n")

In [ ]:
#manual delete. The cookie can be int or hex
# !python3 -m ryu_flow_installer --dpid 1 --delete --cookie 4289248501979854311

[dpid 1] delete(cookie=0x3b8678c981e2b5e7) via cookie(int) -> still present? NO
